In [ ]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from typing import Dict, List, Tuple, Optional
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.core.position import Position
from neuropy.core.session.dataSession import DataSession
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot

from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string, CapturedException

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends # for `_perform_long_short_instantaneous_spike_rate_groups_analysis`
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import SingleBarResult, InstantaneousSpikeRateGroupsComputation, TruncationCheckingResults # for `BatchSessionCompletionHandler`, `AcrossSessionsAggregator`
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_programmatic_figures, batch_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends

from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import *

# Jupyter Widget Interactive
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, fullwidth_path_widget, interactive_pipeline_files

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

# Load Pipeline

In [ ]:
%pdb off
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # DONE, might be the BEST SESSION, good example session with lots of place cells, clean replays, and clear bar graphs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD, 2023-07-14, unsure why still.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE, Added replay selections. Very "jumpy" between the starts and ends of the track.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE, Added replay selections. A TON of putative replays in general, most bad, but some good.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE, added replay selections. Very few (like 12) replays each.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE, okay replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE, very few replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE, one replay each (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE, good. Many good pfs, many good replays. Noticed very strange jumping off the track in the 3D behavior/spikes viewer. Is there something wrong with this session?
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE, good?, replays selected, few
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE, very few replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE, replays selected, quite a few replays but few are very good.


# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25')

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# # # # # Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=['pf_computation', 'pfdt_computation', 'firing_rate_trends',
                                                'pf_dt_sequential_surprise', 
                                            #    'ratemap_peaks_prominence2d',
                                                'position_decoding', 'position_decoding_two_step', 
                                               'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
                                                # 'long_short_inst_spike_rate_groups',
                                                'long_short_endcap_analysis']

curr_active_pipeline: NeuropyPipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=False) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'

if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'cannot load global results: {e}')
        raise


# 6m 1.1s
# 12m 15.6s


# try:
## Post Compute Validate 2023-05-16:
was_updated = BatchSessionCompletionHandler.post_compute_validate(curr_active_pipeline) ## TODO: need to potentially re-save if was_updated
if was_updated:
    try:
        curr_active_pipeline.save_pipeline(saving_mode=saving_mode)
    except Exception as e:
        ## TODO: catch/log saving error and indicate that it isn't saved.
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'ERROR RE-SAVING PIPELINE after update. error: {e}')


curr_active_pipeline.reload_default_computation_functions()
extended_computations_include_includelist=['pf_computation', 'pfdt_computation', 'firing_rate_trends',
    #'pf_dt_sequential_surprise',
     #'ratemap_peaks_prominence2d',
    'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
    #  'long_short_inst_spike_rate_groups',
    'long_short_endcap_analysis'
] # do only specified
force_recompute_global = force_reload
# force_recompute_global = True
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if (len(newly_computed_values) > 0):
    print(f'newly_computed_values: {newly_computed_values}.')
    if (saving_mode.value != 'skip_saving'):
        print(f'Saving global results...')
        try:
            # curr_active_pipeline.global_computation_results.persist_time = datetime.now()
            # Try to write out the global computation function results:
            curr_active_pipeline.save_global_computation_results()
        except Exception as e:
            exception_info = sys.exc_info()
            e = CapturedException(e, exception_info)
            print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
            print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
    else:
        print(f'\n\n!!WARNING!!: changes to global results have been made but they will not be saved since saving_mode.value == "skip_saving"')
        print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
    print(f'no changes in global results.')

# except Exception as e:
#     exception_info = sys.exc_info()
#     e = CapturedException(e, exception_info)
#     print(f'second half threw: {e}')


# 4m 5.2s for inst fr computations
# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, fullwidth_path_widget, interactive_pipeline_files

_pipeline_jupyter_widget = interactive_pipeline_widget(curr_active_pipeline=curr_active_pipeline)
# display(_pipeline_jupyter_widget)
_pipeline_jupyter_widget

In [ ]:
if curr_active_pipeline.updated_since_last_pickle:
	_bak_saving_mode = saving_mode

saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
try:
	curr_active_pipeline.save_pipeline(saving_mode=saving_mode)
	curr_active_pipeline.save_global_computation_results()

except Exception as e:
	## TODO: catch/log saving error and indicate that it isn't saved.
	exception_info = sys.exc_info()
	e = CapturedException(e, exception_info)
	print(f'ERROR RE-SAVING PIPELINE after update. error: {e}')
finally:
	saving_mode = _bak_saving_mode


In [ ]:
def _require_pipeline_has_refined_pfs(curr_active_pipeline, frs_index_inclusion_magnitude:float=0.5):
	""" Refine the LxC/SxC designators using the firing rate index metric:
	"""
	jonathan_firing_rate_analysis_result: JonathanFiringRateAnalysisResult = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis

	## Get global `long_short_fr_indicies_analysis`:
	long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
	long_short_fr_indicies_df = long_short_fr_indicies_analysis_results['long_short_fr_indicies_df']
	jonathan_firing_rate_analysis_result.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=frs_index_inclusion_magnitude)



_require_pipeline_has_refined_pfs(curr_active_pipeline, frs_index_inclusion_magnitude=0.5)

In [ ]:
curr_active_pipeline.clear_display_outputs()	

In [ ]:
curr_active_pipeline.save_global_computation_results()

In [ ]:
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata

hdf5_output_path: Path = curr_active_pipeline.get_output_path().joinpath('pipeline_results.h5').resolve()
print(f'pipeline hdf5_output_path: {hdf5_output_path}')


# Only get files newer than date
newest_file_to_overwrite_date = datetime.now() - timedelta(days=1) # don't overwrite any files more recent than 1 day ago
if hdf5_output_path.exists() and (FilesystemMetadata.get_last_modified_time(hdf5_output_path)<=newest_file_to_overwrite_date):
	# file is folder than the date to overwrite, so overwrite it
	print(f'OVERWRITING (or writing) the file {hdf5_output_path}!')
else:
	print(f'file {hdf5_output_path} is newer than the allowed overwrite date, so it will be skipped.')


# End Run

In [ ]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global 

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_epoch_obj, short_epoch_obj = [Epoch(curr_active_pipeline.sess.epochs.to_dataframe().epochs.label_slice(an_epoch_name)) for an_epoch_name in [long_epoch_name, short_epoch_name]]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global `long_short_fr_indicies_analysis`:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_laps, long_replays, short_laps, short_replays, global_laps, global_replays = [long_short_fr_indicies_analysis_results[k] for k in ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays']]
long_short_fr_indicies_df = long_short_fr_indicies_analysis_results['long_short_fr_indicies_df']

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result.Flat_epoch_time_bins_mean, expected_v_observed_result.Flat_decoder_time_bin_centers, expected_v_observed_result.num_neurons, expected_v_observed_result.num_timebins_in_epoch, expected_v_observed_result.num_total_flat_timebins, expected_v_observed_result.is_short_track_epoch, expected_v_observed_result.is_long_track_epoch, expected_v_observed_result.short_short_diff, expected_v_observed_result.long_long_diff

jonathan_firing_rate_analysis_result: JonathanFiringRateAnalysisResult = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)
neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions(frs_index_inclusion_magnitude=0.5)

## Update long_exclusive/short_exclusive properties with `long_short_fr_indicies_df`
# long_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)
# short_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)

# inst_spike_rate_groups_result: InstantaneousSpikeRateGroupsComputation = curr_active_pipeline.global_computation_results.computed_data.long_short_inst_spike_rate_groups
# custom_InstSpikeRateTrends_df = inst_spike_rate_groups_result.all_incl_endPlatforms_InstSpikeRateTrends_df



In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper

fig_1c_figures_out_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.get_refined_track_exclusive_aclus(), n_max_page_rows=20, write_vector_format=False, write_png=False, show_only_refined_cells=True, disable_top_row=True)


In [ ]:
## all cells:
fig_1c_figures_all_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=None, n_max_page_rows=20, write_vector_format=False, write_png=False, show_only_refined_cells=False, disable_top_row=True)

In [ ]:
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df.copy()

np.sum(np.logical_xor(neuron_replay_stats_df['is_refined_LxC'], neuron_replay_stats_df['is_refined_SxC']))

In [ ]:
np.sum(np.logical_and(neuron_replay_stats_df['is_refined_LxC'], neuron_replay_stats_df['is_refined_SxC']))

In [ ]:
# uses global's 'extended_stats' results for relative entropy (surprise) analyses:

import tables as tb


# {
# 't': Tuple[float, float],
# 'snapshots': Tuple[PlacefieldSnapshot, PlacefieldSnapshot],
# 'relative_entropy_result_dict': {
# 		'long_short_rel_entr_curve': np.array,
# 		...
# 		'jensen_shannon_distance': np.array,
# 	} # 5 items
# } # 3 items
# Dict[float, PlacefieldSnapshot]

def relative_entropy_to_h5(global_results, file_path='output/test_relative_entropy_numpy_arrays.h5'):
	print(f'relative_entropy_to_h5(...)')

	active_extended_stats = global_results['extended_stats']
	active_relative_entropy_results = active_extended_stats['pf_dt_sequential_surprise'] # DynamicParameters
	post_update_times: np.ndarray = active_relative_entropy_results['post_update_times'] # (4152,) = (n_post_update_times,)
	snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
	time_intervals: np.ndarray = active_relative_entropy_results['time_intervals']
	long_short_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['long_short_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
	short_long_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['short_long_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
	flat_relative_entropy_results: np.ndarray = active_relative_entropy_results['flat_relative_entropy_results'] # (149, 63) - (nSnapshots, nXbins)
	flat_jensen_shannon_distance_results: np.ndarray = active_relative_entropy_results['flat_jensen_shannon_distance_results'] # (149, 63) - (nSnapshots, nXbins)
	flat_jensen_shannon_distance_across_all_positions: np.ndarray = np.sum(np.abs(flat_jensen_shannon_distance_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)
	flat_surprise_across_all_positions: np.ndarray = np.sum(np.abs(flat_relative_entropy_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)
	
	# Create an HDF5 file
	with tb.open_file(file_path, mode="w") as hdf5_file:
		print(f'trying to write to "{file_path}"...')
		# Create groups for organization
		root = hdf5_file.root
		group = hdf5_file.create_group(root, 'relative_entropy_results')
		
		# Store np.ndarrays in the HDF5 file
		hdf5_file.create_array(group, 'post_update_times', post_update_times)
		hdf5_file.create_array(group, 'time_intervals', time_intervals)
		hdf5_file.create_array(group, 'long_short_rel_entr_curves_frames', long_short_rel_entr_curves_frames)
		hdf5_file.create_array(group, 'short_long_rel_entr_curves_frames', short_long_rel_entr_curves_frames)
		hdf5_file.create_array(group, 'flat_relative_entropy_results', flat_relative_entropy_results)
		hdf5_file.create_array(group, 'flat_jensen_shannon_distance_results', flat_jensen_shannon_distance_results)
		hdf5_file.create_array(group, 'flat_jensen_shannon_distance_across_all_positions', flat_jensen_shannon_distance_across_all_positions)
		hdf5_file.create_array(group, 'flat_surprise_across_all_positions', flat_surprise_across_all_positions)
	
	print(f'\t done!')
	

relative_entropy_to_h5(global_results, file_path='output/test_relative_entropy_numpy_arrays.h5')

In [ ]:
curr_active_pipeline.export_pipeline_to_h5()


In [ ]:
snapshot_differences_result_dict

flat_jensen_shannon_distance_results

In [ ]:
active_relative_entropy_results
# type(active_relative_entropy_results)

In [ ]:
neuron_replay_stats_df: pd.DataFrame = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
neuron_replay_stats_df


In [ ]:
# Test classifying various x-positions as belonging to outside the outside_maze, the track_endcaps, or the track_body
from enum import Enum

class TrackPositionClassification(Enum):
    OUTSIDE_MAZE = "outside_maze"
    TRACK_ENDCAPS = "track_endcaps"
    TRACK_BODY = "track_body"


neuron_replay_stats_df: pd.DataFrame = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
short_pf_peak_x = neuron_replay_stats_df.short_pf_peak_x
long_pf_peak_x = neuron_replay_stats_df.long_pf_peak_x

In [ ]:
rate_remapping_df[np.isin(rate_remapping_df.index, significant_distant_remapping_endcap_aclus)]

In [ ]:
inst_spike_rate_groups_result: InstantaneousSpikeRateGroupsComputation = curr_active_pipeline.global_computation_results.computed_data.long_short_inst_spike_rate_groups
# custom_InstSpikeRateTrends_df = inst_spike_rate_groups_result.all_incl_endPlatforms_InstSpikeRateTrends_df
# if not hasattr(inst_spike_rate_groups_result, 'all_incl_endPlatforms_InstSpikeRateTrends_df'):
# 	inst_spike_rate_groups_result.all_incl_endPlatforms_InstSpikeRateTrends_df = pd.DataFrame()

In [ ]:
# 2023-09-14 - Find cells outside the bounds of the short track
# Modifies the `jonathan_firing_rate_analysis_result.neuron_replay_stats_df` in-place instead of creating a copy:
# neuron_replay_stats_df = deepcopy(jonathan_firing_rate_analysis_result.neuron_replay_stats_df)
neuron_replay_stats_df: pd.DataFrame = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
# Extract the peaks of the long placefields to find ones that have peaks outside the boundaries
# long_pf_peaks = neuron_replay_stats_df[neuron_replay_stats_df['has_long_pf']]['long_pf_peak_x'] - 150.0 # this shift of 150.0 is to center the midpoint of the track at 0. 
# # display(long_pf_peaks)
# is_left_cap = (long_pf_peaks < -72.0)
# is_right_cap = (long_pf_peaks > 72.0)
# is_either_cap =  np.logical_or(is_left_cap, is_right_cap)

# # Adds ['is_long_peak_left_cap', 'is_long_peak_right_cap', 'is_long_peak_either_cap'] columns: 
# neuron_replay_stats_df['is_long_peak_left_cap'] = False
# neuron_replay_stats_df['is_long_peak_right_cap'] = False
# neuron_replay_stats_df.loc[is_left_cap.index, 'is_long_peak_left_cap'] = is_left_cap # True
# neuron_replay_stats_df.loc[is_right_cap.index, 'is_long_peak_right_cap'] = is_right_cap # True
# neuron_replay_stats_df['is_long_peak_either_cap'] = np.logical_or(neuron_replay_stats_df['is_long_peak_left_cap'], neuron_replay_stats_df['is_long_peak_right_cap'])
# adds ['LS_pf_peak_x_diff'] column
# neuron_replay_stats_df['LS_pf_peak_x_diff'] = neuron_replay_stats_df['long_pf_peak_x'] - neuron_replay_stats_df['short_pf_peak_x']
neuron_replay_stats_df.is_long_peak_either_cap

## Extract just the endcap cells:
cap_cells_df = neuron_replay_stats_df[np.logical_and(neuron_replay_stats_df['has_long_pf'], neuron_replay_stats_df['is_long_peak_either_cap'])]
cap_aclus = cap_cells_df.index
num_total_endcap_cells = len(cap_aclus)
display(num_total_endcap_cells)

# "Disppearing" cells fall below the 1Hz firing criteria on the short track:
disappearing_endcap_cells_df = cap_cells_df[np.logical_not(cap_cells_df['has_short_pf'])]
disappearing_endcap_aclus = disappearing_endcap_cells_df.index
num_disappearing_endcap_cells = len(disappearing_endcap_aclus)
print(f'num_disappearing_endcap_cells/num_total_endcap_cells: {num_disappearing_endcap_cells}/{num_total_endcap_cells}')

non_disappearing_endcap_cells_df = cap_cells_df[cap_cells_df['has_short_pf']] # "non_disappearing" cells are those with a placefield on the short track as well
num_non_disappearing_endcap_cells = len(non_disappearing_endcap_cells_df)
print(f'num_non_disappearing_endcap_cells/num_total_endcap_cells: {num_non_disappearing_endcap_cells}/{num_total_endcap_cells}')

# display(non_disappearing_endcap_cells_df)
# non_disappearing_endcap_cells_df['LS_pf_peak_x_diff'] = non_disappearing_endcap_cells_df['long_pf_peak_x'] - non_disappearing_endcap_cells_df['short_pf_peak_x']
# display(non_disappearing_endcap_cells_df)

# Classify the non_disappearing cells into two groups:
# 1. Those that exhibit significant remapping onto somewhere else on the track
non_disappearing_endcap_cells_df['has_significant_distance_remapping'] = (np.abs(non_disappearing_endcap_cells_df['LS_pf_peak_x_diff']) >= 40.0) # The most a placefield could translate intwards would be (35 + (pf_width/2.0)) I think.
num_significant_position_remappping_endcap_cells = len(non_disappearing_endcap_cells_df[non_disappearing_endcap_cells_df['has_significant_distance_remapping'] == True])
print(f'num_significant_position_remappping_endcap_cells/num_non_disappearing_endcap_cells: {num_significant_position_remappping_endcap_cells}/{num_non_disappearing_endcap_cells}')

# 2. Those that seem to remain where they were on the long track, perhaps being "sampling-clipped" or translated adjacent to the platform. These two subcases can be distinguished by a change in the placefield's length (truncated cells would be a fraction of the length, although might need to account for scaling with new track length)
minorly_changed_endcap_cells_df = non_disappearing_endcap_cells_df[non_disappearing_endcap_cells_df['has_significant_distance_remapping'] == False]

non_disappearing_endcap_cells_df
# endcaps:
significant_distant_remapping_endcap_aclus = non_disappearing_endcap_cells_df[non_disappearing_endcap_cells_df['has_significant_distance_remapping']].index.to_numpy() # Int64Index([3, 5, 7, 11, 14, 38, 41, 53, 57, 61, 62, 75, 78, 79, 82, 83, 85, 95, 98, 100, 102], dtype='int64')
significant_distant_remapping_endcap_aclus
minor_remapping_endcap_aclus = minorly_changed_endcap_cells_df.index.to_numpy()
minor_remapping_endcap_aclus
jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
significant_distant_remapping_endcap_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.isin(jonathan_firing_rate_analysis_result.neuron_replay_stats_df.index, significant_distant_remapping_endcap_aclus)]
significant_distant_remapping_endcap_df

In [ ]:
# Will these remapped cells be included in replays after the delta?
num_short_replays = non_disappearing_endcap_cells_df['short_num_replays']
short_replay_mean_fr_Hz = non_disappearing_endcap_cells_df['replay_diff']


## 2023-09-20 - Plot the significantly remapping cells using the lower-level `plot_short_v_long_pf1D_comparison` function

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_short_v_long_pf1D_comparison
from neuropy.utils.matplotlib_helpers import perform_update_title_subtitle
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import add_vertical_track_bounds_lines, add_track_shapes
from neuropy.plotting.ratemaps import plot_ratemap_1D
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import LongShortDisplayConfigManager

curr_active_pipeline.reload_default_display_functions()

graphics_output_dict = {}
active_context: IdentifyingContext = curr_active_pipeline.get_session_context()

long_short_display_config_manager = LongShortDisplayConfigManager()
long_epoch_config = long_short_display_config_manager.long_epoch_config.as_pyqtgraph_kwargs()
short_epoch_config = long_short_display_config_manager.short_epoch_config.as_pyqtgraph_kwargs()

long_epoch_matplotlib_config = long_short_display_config_manager.long_epoch_config.as_matplotlib_kwargs()
short_epoch_matplotlib_config = long_short_display_config_manager.short_epoch_config.as_matplotlib_kwargs()

shared_kwargs = dict(pad=1, cmap='hsv', active_context=curr_active_pipeline.get_session_context(), plot_zero_baselines=True, skip_figure_titles=True, use_flexitext_titles=True, flat_stack_mode=False)
top_level_shared_kwargs = dict(should_plot_vertical_track_bounds_lines=True, sortby='peak_long')
# top_level_shared_kwargs = dict(should_plot_vertical_track_bounds_lines=False, should_plot_linear_track_shapes=True) # Renders the linear track shape on the maze. Assumes `flat_stack_mode=True`


# # flat_stack_mode: all placefields are stacked up (z-wise) on top of each other on a single axis with no offsets:
# shared_kwargs = dict(pad=1, active_context=curr_active_pipeline.get_session_context(), plot_zero_baselines=True, skip_figure_titles=True, use_flexitext_titles=True, flat_stack_mode=True)
# top_level_shared_kwargs = dict(should_plot_vertical_track_bounds_lines=False, should_plot_linear_track_shapes=True) # Renders the linear track shape on the maze. Assumes `flat_stack_mode=True`

In [ ]:
# 2023-09-21 - Plot All
graphics_output_dict = graphics_output_dict | fig_remapping_cells(curr_active_pipeline)

In [ ]:
plt.suptitle('Translation Remapping Cells')

In [ ]:
fig_surprise_results

In [ ]:
# long_results = curr_active_pipeline.computation_results['maze1_PYR'].computed_data
# short_results = curr_active_pipeline.computation_results['maze2_PYR'].computed_data
# curr_any_context_neurons = _find_any_context_neurons(*[curr_active_pipeline.computation_results[k].computed_data.pf1D.ratemap.neuron_ids for k in ['maze1_PYR', 'maze2_PYR']])
# (fig_long_pf_1D, ax_long_pf_1D, long_sort_ind, long_neurons_colors_array), (fig_short_pf_1D, ax_short_pf_1D, short_sort_ind, short_neurons_colors_array) = plot_short_v_long_pf1D_comparison(long_results, short_results, disappearing_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Disappearing Cells", subtitle_string=None, **top_level_shared_kwargs)
graphics_output_dict['disappearing_endcap_aclus'] = curr_active_pipeline.display('_display_short_long_pf1D_comparison', curr_active_pipeline.get_session_context(), included_any_context_neuron_ids=disappearing_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Disappearing Cells", subtitle_string=None, **top_level_shared_kwargs)

In [ ]:
# (fig_long_pf_1D, ax_long_pf_1D, long_sort_ind, long_neurons_colors_array), (fig_short_pf_1D, ax_short_pf_1D, short_sort_ind, short_neurons_colors_array) = plot_short_v_long_pf1D_comparison(long_results, short_results, significant_distant_remapping_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Significant Distance Remapping Cells", subtitle_string="1D Placefields", **top_level_shared_kwargs)
graphics_output_dict['significant_distant_remapping_endcap_aclus'] = curr_active_pipeline.display('_display_short_long_pf1D_comparison', curr_active_pipeline.get_session_context(), included_any_context_neuron_ids=significant_distant_remapping_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Significant Distance Remapping Cells", subtitle_string="1D Placefields", **top_level_shared_kwargs)

In [ ]:
# (fig_long_pf_1D, ax_long_pf_1D, long_sort_ind, long_neurons_colors_array), (fig_short_pf_1D, ax_short_pf_1D, short_sort_ind, short_neurons_colors_array) = plot_short_v_long_pf1D_comparison(long_results, short_results, trivially_remapping_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Trivially Remapping Cells", subtitle_string="1D Placefields", **top_level_shared_kwargs)
graphics_output_dict['trivially_remapping_endcap_aclus'] = curr_active_pipeline.display('_display_short_long_pf1D_comparison', curr_active_pipeline.get_session_context(), included_any_context_neuron_ids=trivially_remapping_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Trivially Remapping Cells", subtitle_string="1D Placefields", **top_level_shared_kwargs)

In [ ]:
## 2023-09-18 - Shows the cells that exhibit simple remapping after delta
graphics_output_dict = curr_active_pipeline.display('_display_short_long_pf1D_comparison', curr_active_pipeline.get_session_context(), included_any_context_neuron_ids=None, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, **top_level_shared_kwargs)

In [ ]:
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import fig_example_nontopo_remap

graphics_output_dict = fig_example_nontopo_remap(curr_active_pipeline)

In [ ]:
curr_active_pipeline.export_pipeline_to_h5('output/2023_09_26_new_pipeline_test.h5')
# ERROR: encountered exception !! 'InstantaneousSpikeRateGroupsComputation' object has no attribute 'all_incl_endPlatforms_InstSpikeRateTrends_df' ::::: (<class 'AttributeError'>, AttributeError("'InstantaneousSpikeRateGroupsComputation' object has no attribute 'all_incl_endPlatforms_InstSpikeRateTrends_df'"), <traceback object at 0x000001E5A3D3D040>) while trying to build the session HDF output.
# AttributeError: 'SpikeRateTrends' object has no attribute 'included_neuron_ids'
# _neuron_replay_stats_df - TypeError: Cannot serialize the column [track_membership] because its data contents are not [string] but [mixed] object dtype

In [ ]:
%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

graphics_output_dict = curr_active_pipeline.display('_display_short_long_pf1D_scalar_overlap_comparison', active_identifying_session_ctx, save_figure=False)
# fig, axs, plot_data = graphics_output_dict['fig'], graphics_output_dict['axs'], graphics_output_dict['plot_data']

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
# _display_long_short_expected_v_observed_firing_rate
graphics_output_dict = curr_active_pipeline.display('_display_long_short_expected_v_observed_firing_rate', curr_active_pipeline.sess.get_context(), included_neuron_IDs=significant_distant_remapping_endcap_aclus)

In [ ]:
_temp_force_recompute=True

In [ ]:
## Recompute 'long_short_fr_indicies_analyses'
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_computations

curr_active_pipeline.reload_default_computation_functions()

# _temp_force_recompute=True
_temp_force_recompute=False
# extended_computations_include_includelist = ['_perform_time_dependent_placefield_computation', 'long_short_fr_indicies_analyses', 'pf_dt_sequential_surprise', 'short_long_pf_overlap_analyses'] # do only specifiedl
extended_computations_include_includelist = ['jonathan_firing_rate_analysis', 'short_long_pf_overlap_analyses']
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=_temp_force_recompute, debug_print=False)
newly_computed_values


In [ ]:
jonathan_firing_rate_analysis_result.rdf.rdf

In [ ]:
jonathan_firing_rate_analysis_result.irdf.irdf

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
jonathan_firing_rate_analysis_result

In [ ]:
curr_long_short_fr_indicies_analysis_bak = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']

curr_long_short_fr_indicies_analysis = curr_active_pipeline.global_computation_results.computed_data.pop('long_short_fr_indicies_analysis')

print(list(curr_long_short_fr_indicies_analysis.keys())) # ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays', 'long_non_replays', 'short_non_replays', 'global_non_replays', 'long_mean_non_replays_frs', 'short_mean_non_replays_frs', 'long_mean_non_replays_all_frs', 'short_mean_non_replays_all_frs', 'non_replays_frs_index', 'long_mean_non_replays_all_inst_frs', 'short_mean_non_replays_all_inst_frs', 'non_replays_inst_frs_index', 'active_context']

print_keys_if_possible('curr_long_short_fr_indicies_analysis', curr_long_short_fr_indicies_analysis)

In [ ]:
curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']

# 2023-09-12 - Assemble all neuron-level properties:

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies

joined_neruon_fri_df = build_merged_neuron_firing_rate_indicies(curr_active_pipeline, enable_display_intermediate_results=False)
joined_neruon_fri_df

In [ ]:
print(list(joined_neruon_fri_df.columns)) # ['aclu', 'lsfria_laps_frs_index', 'lsfria_laps_inst_frs_index', 'lsfria_replays_frs_index', 'lsfria_replays_inst_frs_index', 'lsfria_non_replays_frs_index', 'lsfria_non_replays_inst_frs_index', 'jfra_long_pf_peak_x', 'jfra_has_long_pf', 'jfra_short_pf_peak_x', 'jfra_has_short_pf', 'jfra_has_na', 'jfra_track_membership', 'jfra_long_non_replay_mean', 'jfra_short_non_replay_mean', 'jfra_non_replay_diff', 'jfra_long_replay_mean', 'jfra_short_replay_mean', 'jfra_replay_diff', 'jfra_long_mean', 'jfra_short_mean', 'jfra_mean_diff', 'jfra_neuron_IDX', 'jfra_num_replays', 'jfra_long_num_replays', 'jfra_short_num_replays', 'jfra_neuron_type', 'lspd_laps', 'lspd_replays', 'lspd_skew', 'lspd_max_axis_distance_from_center', 'lspd_distance_from_center', 'lspd_has_considerable_remapping']

# Computing consolidated `long_short_fr_indicies_df`

In [ ]:
# 'long_short_fr_indicies_analysis'
curr_long_short_fr_indicies_analysis = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
_curr_aclus = list(curr_long_short_fr_indicies_analysis['laps_frs_index'].keys()) # extract one set of keys for the aclus
_curr_frs_indicies_dict = {k:v.values() for k,v in curr_long_short_fr_indicies_analysis.items() if k in ['laps_frs_index', 'laps_inst_frs_index', 'replays_frs_index', 'replays_inst_frs_index', 'non_replays_frs_index', 'non_replays_inst_frs_index']} # extract the values
long_short_fr_indicies_df = pd.DataFrame(_curr_frs_indicies_dict, index=_curr_aclus)
long_short_fr_indicies_df

In [ ]:
long_short_fr_indicies_df

In [ ]:
curr_active_pipeline.reload_default_computation_functions()
curr_active_pipeline.perform_specific_computation('long_short_fr_indicies_analyses')


In [ ]:
# all_results_dict

In [ ]:
# ax = long_short_fr_indicies_df.plot.scatter(x='non_replays_inst_frs_index' , y='replays_inst_frs_index', title='Replays v. Non-replay Firing Rate Index Comparison')
# long_short_fr_indicies_df.plot.scatter(x='laps_inst_frs_index' , y='replays_inst_frs_index', title='Replays v. Laps Firing Rate Index Comparison', ax=ax)

ax = long_short_fr_indicies_df.plot.scatter(x='non_replays_frs_index' , y='replays_frs_index', title='Replays v. Non-replay Firing Rate Index Comparison')
long_short_fr_indicies_df.plot.scatter(x='laps_frs_index' , y='replays_frs_index', title='Replays v. Laps Firing Rate Index Comparison', ax=ax)


In [ ]:
from pandas.plotting import scatter_matrix

# scatter_matrix(long_short_fr_indicies_df, figsize=(10, 10))
scatter_matrix(joined_neruon_fri_df, figsize=(10, 10))


## Plots the RateRemappingFiringRateIndex Number Line Figure


In [ ]:
# ## Extract rr_* variables from rate_remapping_df
# rr_aclus = rate_remapping_df.index.values
# rr_laps, rr_replays, rr_skew, rr_neuron_type = [rate_remapping_df[n].values for n in ['laps', 'replays', 'skew', 'neuron_type']]


## Extract rr_* variables from joined_neruon_fri_df
rr_aclus = joined_neruon_fri_df.index.values
rr_laps, rr_replays, rr_skew, rr_neuron_type = [joined_neruon_fri_df[n].values for n in ['lspd_laps', 'lspd_replays', 'lspd_skew', 'jfra_neuron_type']]

## Display:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_rr_aclu
n_debug_limit = 10
fig, axs, sort_indicies = plot_rr_aclu([str(aclu) for aclu in rr_aclus[:n_debug_limit]], rr_laps=rr_laps[:n_debug_limit], rr_replays=rr_replays[:n_debug_limit], rr_neuron_types=rr_neuron_type[:n_debug_limit])


In [ ]:
fri_index_dicts_list = [dict(x=rr_laps, marker=r'$\theta$', markersize=10, color='black', label='rr_laps'),
	dict(x=rr_replays, marker='o', markersize=10, fillstyle='none', color='black', label='rr_replays'),
]

In [ ]:

## Display Paginated multi-plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_rr_aclu
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import RateRemappingPaginatedFigureController
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_rr_pagination_controller = RateRemappingPaginatedFigureController.init_from_rr_data(rr_aclus, rr_laps, rr_replays, rr_neuron_type, max_subplots_per_page=30, a_name='TestRateRemappingPaginatedFigureController', active_context=active_identifying_session_ctx)
a_paginator = _out_rr_pagination_controller.plots_data.paginator

# 2023-09-28 - InstFrRate-based classification of LxC and SxC
2023-09-28 10:39am: We looked at the results and determined that it was much worse than the placefield critiera I had previously. This analysis should be used as a secondary refinement for the existing placefield-based LxC/SxC exclusion critiera. It includes information about non-lap endcap activity that the other analysis omits. Can be used with a threshold.

In [ ]:
## Plot the selected LxC/SxC cells cells on the PhoJonathan plots
curr_active_pipeline.reload_default_display_functions()
refined_LxC_aclus = long_exclusive.get_refined_track_exclusive_aclus()
print(f'refined_LxC_aclus: {refined_LxC_aclus}')
if len(refined_LxC_aclus):
	_out_LxC = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', n_max_plot_rows=10, save_figure=False, included_unit_neuron_IDs=refined_LxC_aclus) # , included_unit_neuron_IDs=[4, 58]

refined_SxC_aclus = short_exclusive.get_refined_track_exclusive_aclus()
print(f'refined_SxC_aclus: {refined_SxC_aclus}')
if len(refined_SxC_aclus):
	_out_SxC = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', n_max_plot_rows=10, save_figure=False, included_unit_neuron_IDs=refined_SxC_aclus) # , included_unit_neuron_IDs=[4, 58]


In [ ]:

loaded_df = pd.read_csv(Path(r'W:\Data\neuron_replay_stats_table_2023-09-29-GL.csv'))
loaded_df

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import _fr_index
instantaneous_time_bin_size_seconds = 0.1
owning_pipeline_reference = curr_active_pipeline
long_epoch_obj, short_epoch_obj = [Epoch(owning_pipeline_reference.sess.epochs.to_dataframe().epochs.label_slice(an_epoch_name)) for an_epoch_name in [long_epoch_name, short_epoch_name]]

# non_running_periods = Epoch.from_PortionInterval(owning_pipeline_reference.sess.laps.as_epoch_obj().to_PortionInterval().complement())
non_replay_periods: Epoch = Epoch(Epoch.from_PortionInterval(owning_pipeline_reference.sess.replay.epochs.to_PortionInterval().complement()).time_slice(t_start=long_epoch_obj.t_start, t_stop=short_epoch_obj.t_stop).to_dataframe()[:-1]) #[:-1] # any period except the replay ones, drop the infinite last entry
long_only_non_replay_periods: Epoch  = non_replay_periods.time_slice(t_start=long_epoch_obj.t_start, t_stop=long_epoch_obj.t_stop) # any period except the replay ones
short_only_non_replay_periods: Epoch  = non_replay_periods.time_slice(t_start=short_epoch_obj.t_start, t_stop=short_epoch_obj.t_stop) # any period except the replay ones

# ~20sec computation
long_custom_InstSpikeRateTrends: SpikeRateTrends = SpikeRateTrends.init_from_spikes_and_epochs(spikes_df=deepcopy(owning_pipeline_reference.sess.spikes_df),
                                                                                        filter_epochs=long_only_non_replay_periods,
                                                                                        #    included_neuron_ids=long_exclusive.track_exclusive_aclus,
                                                                                        instantaneous_time_bin_size_seconds=instantaneous_time_bin_size_seconds)

short_custom_InstSpikeRateTrends: SpikeRateTrends = SpikeRateTrends.init_from_spikes_and_epochs(spikes_df=deepcopy(owning_pipeline_reference.sess.spikes_df),
                                                                                        filter_epochs=short_only_non_replay_periods,
                                                                                        #    included_neuron_ids=long_exclusive.track_exclusive_aclus,
                                                                                        instantaneous_time_bin_size_seconds=instantaneous_time_bin_size_seconds)

# # Note custom_InstSpikeRateTrends is global, not really needed:
# global_custom_InstSpikeRateTrends: SpikeRateTrends = SpikeRateTrends.init_from_spikes_and_epochs(spikes_df=deepcopy(owning_pipeline_reference.sess.spikes_df),
#                                                                                            filter_epochs=non_replay_periods,
#                                                                                         #    included_neuron_ids=long_exclusive.track_exclusive_aclus,
#                                                                                            instantaneous_time_bin_size_seconds=instantaneous_time_bin_size_seconds)

## Determine the included percentiles for `joined_neruon_fri_df`
custom_InstSpikeRateTrends_df = pd.DataFrame({'aclu': long_custom_InstSpikeRateTrends.included_neuron_ids, 'long_inst_fr': long_custom_InstSpikeRateTrends.cell_agg_inst_fr_list, 'short_inst_fr': short_custom_InstSpikeRateTrends.cell_agg_inst_fr_list}) #
# custom_InstSpikeRateTrends_df['global_inst_fr'] = global_custom_InstSpikeRateTrends.cell_agg_inst_fr_list

# Compute the single-dimensional firing rate index for the custom epochs and add it as a column to the dataframe:
custom_InstSpikeRateTrends_df['custom_frs_index'] = _fr_index(long_fr=long_custom_InstSpikeRateTrends.cell_agg_inst_fr_list, short_fr=short_custom_InstSpikeRateTrends.cell_agg_inst_fr_list)

# # Calculate 10th and 90th percentiles
# lower_bound = custom_InstSpikeRateTrends_df['custom_frs_index'].quantile(0.10)
# upper_bound = custom_InstSpikeRateTrends_df['custom_frs_index'].quantile(0.90)

# # Filter rows
# bottom_10_percent = custom_InstSpikeRateTrends_df[custom_InstSpikeRateTrends_df['custom_frs_index'] <= lower_bound]
# top_10_percent = custom_InstSpikeRateTrends_df[custom_InstSpikeRateTrends_df['custom_frs_index'] >= upper_bound]

# # Extract the aclus from these rows:
# LxC_10_percent_aclus = top_10_percent.aclu.to_numpy()
# SxC_10_percent_aclus = bottom_10_percent.aclu.to_numpy()

# print(f'LxC_10_percent_aclus: {LxC_10_percent_aclus}')
# print(f'SxC_10_percent_aclus: {SxC_10_percent_aclus}')

In [ ]:
## Really want to penalize for any firing on the opposite track.

# 2023-09-18 - Computation Validators and Computations as Needed


In [ ]:
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper, print_widget_hierarchy

In [ ]:
app = pg.mkQApp()
app
app.topLevelWindows()

top_level_windows = []

In [ ]:
top_level_windows = TopLevelWindowHelper.top_level_windows(app, only_visible=True)
top_level_windows # IndexedOrderedDict([('DockClassWindow', <PyQt5.QtGui.QWindow object at 0x000001F7383E9670>), ('qt_splithandle_Window', <PyQt5.QtGui.QWindow object at 0x000001F7383E9280>), ('PhoDockAreaContainingWindowClassWindow', <PyQt5.QtGui.QWindow object at 0x000001F7383E9550>), ('DockAreaClassWindow', <PyQt5.QtGui.QWindow object at 0x000001F7383E98B0>), ('VContainerClassWindow', <PyQt5.QtGui.QWindow object at 0x000001F7383E9430>), ('DockLabelClassWindow', <PyQt5.QtGui.QWindow object at 0x000001F7383D8AF0>), ('QWidgetClassWindow', <PyQt5.QtGui.QWindow object at 0x000001F7370CD280>), ('MainWindowClassWindow', <PyQt5.QtGui.QWindow object at 0x000001F7370CD160>), ('QFrameClassWindow', <PyQt5.QtGui.QWindow object at 0x000001F7370CD1F0>)])


In [ ]:
# 'Phodockareacontainingwindowclasswindow'

for a_win_key, a_win in top_level_windows.items():
	print(f'{a_win}')
	# print(f'\t{a_win.title()}')
	# print(f'\t{type(a_win)}')
	print_widget_hierarchy(a_win, indent='\t\t')
	

In [ ]:
searchType = 'PhoDockAreaContainingWindowClassWindow'
top_level_windows_with_superclass_list = [a_widget for a_widget in top_level_windows if isinstance(a_widget, (searchType))]
top_level_windows_with_superclass_list

In [ ]:
'PhoDockAreaContainingWindowClassWindow'

In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_computations

curr_active_pipeline.reload_default_computation_functions()


In [ ]:

extended_computations_include_includelist = ['pf_computation', 'pfdt_computation', 'long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding', 
    '_perform_time_dependent_pf_sequential_surprise_computation',
     'long_short_rate_remapping',
     'long_short_inst_spike_rate_groups'
    ] # do only specifiedl , 'long_short_rate_remapping'
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=False, debug_print=False)
newly_computed_values

In [ ]:
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=['_perform_pf_find_ratemap_peaks_peak_prominence2d_computation'], included_computation_filter_names=['maze1', 'maze2', 'maze'], include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=False, debug_print=False)


In [ ]:
'_perform_long_short_endcap_analysis'
curr_active_pipeline.reload_default_computation_functions()
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=['_perform_long_short_endcap_analysis'], include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=False, debug_print=False)
newly_computed_values

In [ ]:
curr_active_pipeline.updated_since_last_pickle

In [ ]:
newly_computed_values

In [ ]:
newly_computed_values

In [ ]:
rate_remapping_df

In [ ]:
rate_remapping_df[['laps',	'replays',	'skew',	'max_axis_distance_from_center', 'distance_from_center', 'has_considerable_remapping']] # drops ['neuron_type', 'render_color']

In [ ]:
from neuropy.utils.mixins.HDF5_representable import HDF_Converter

file_path = 'output/test_rate_remapping_new.h5'

session_context = curr_active_pipeline.get_session_context() 
session_group_key: str = "/" + session_context.get_description(separator="/", include_property_names=False) # 'kdiba/gor01/one/2006-6-08_14-26-15'
session_uid: str = session_context.get_description(separator="|", include_property_names=False)
## Global Computations
a_global_computations_group_key: str = f"{session_group_key}/global_computations"
with tb.open_file(file_path, mode='w') as f:
    a_global_computations_group = f.create_group(session_group_key, 'global_computations', title='the result of computations that operate over many or all of the filters in the session.', createparents=True)


# rate_remapping_df
# rate_remapping_df = rate_remapping_df[['laps',	'replays',	'skew',	'max_axis_distance_from_center',	'distance_from_center', 	'has_considerable_remapping']]
# # active_context = kwargs.pop('active_context', None) # TODO: requiring that the caller pass active_context isn't optimal
# active_context = session_context
# rate_remapping_df = HDF_Converter.prepare_neuron_indexed_dataframe_for_hdf(rate_remapping_df, active_context=active_context, aclu_column_name=None)
# # rate_remapping_df.to_hdf(file_path, key=f'{key}/rate_remapping_df', format='table', data_columns=True)
curr_long_short_rr.to_hdf(file_path, key=f'{a_global_computations_group_key}/rate_remapping', active_context=curr_active_pipeline.get_session_context())

In [ ]:
# curr_active_pipeline.global_computation_results_pickle_path
# curr_active_pipeline.pickle_path
curr_active_pipeline.get_output_path()

In [ ]:
outman = curr_active_pipeline.get_output_manager()
figure_output_path = outman.get_figure_save_file_path(curr_active_pipeline.get_session_context(), make_folder_if_needed=False)

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_1d_placecell_validations

# curr_active_pipeline.plotting_config
active_session_configuration_name = global_epoch_name
active_config = curr_active_pipeline.active_configs[active_session_configuration_name]

_out = plot_1d_placecell_validations(global_pf1D, active_config.plotting_config, modifier_string='lap_only', should_save=False)

In [ ]:
# _out.figures[0].show()
plt.show()

In [ ]:
curr_active_pipeline.to_hdf('output/test_pipeline_fixed.h5', key='/')
# TypeError: objects of type ``list`` are not supported in this context, sorry; supported objects are: NumPy array, record or scalar
# for flattened_spike
# .infer_objects()

# curr_active_pipeline.export_pipeline_to_h5()
# self.to_hdf(file_path=hdf5_output_path, key="/")

In [ ]:
from neuropy.core.session.dataSession import DataSession

## Test saaving sessions
sess: DataSession = deepcopy(curr_active_pipeline.sess)
sess

In [ ]:
sess.to_hdf('output/test_sess.h5', key='/')

In [ ]:
sess.laps
# sess.ripples
sess.pbe

In [ ]:
# historical_snapshots


In [ ]:
# Make sure pickling is okay with pf1D_dt
global_pf1D_dt.to_hdf('output/test_global_pf1D_dt.h5', key='global_pf1D_dt')

In [ ]:
override_pickle_path: Path = Path('output/test_pipeline.pkl').resolve()
override_pickle_path

In [ ]:
active_extended_stats = curr_active_pipeline.computation_results['maze'].computed_data['extended_stats']
active_extended_stats

In [ ]:
active_relative_entropy_results = active_extended_stats['relative_entropy_analyses']
post_update_times = active_relative_entropy_results['post_update_times']
snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
time_intervals = active_relative_entropy_results['time_intervals']
long_short_rel_entr_curves_frames = active_relative_entropy_results['long_short_rel_entr_curves_frames']
short_long_rel_entr_curves_frames = active_relative_entropy_results['short_long_rel_entr_curves_frames']
flat_relative_entropy_results = active_relative_entropy_results['flat_relative_entropy_results']
flat_jensen_shannon_distance_results = active_relative_entropy_results['flat_jensen_shannon_distance_results']

In [ ]:
type(active_relative_entropy_results)


In [ ]:
global_pf1D_dt.plot_ratemaps_1D()

In [ ]:
NeuronExtendedIdentityTuple(shank=10, cluster=10, id=87),

In [ ]:
list(long_pf1D.included_neuron_IDs).index(87)

In [ ]:
long_pf1D.plot_all(87)

In [ ]:
long_pf1D.plotRaw_v_time(87)

In [ ]:
curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison')

In [ ]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=False)

In [ ]:
from typing import Optional
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from neuropy.core.neurons import NeuronType
from neuropy.utils.mixins.HDF5_representable import HDF_Converter

# Generate the unique aclus
_neuron_replay_stats_df = deepcopy(jonathan_firing_rate_analysis_result.neuron_replay_stats_df)
_neuron_replay_stats_df

# _neuron_replay_stats_df = prepare_neuron_indexed_dataframe_for_hdf(_neuron_replay_stats_df, active_context=curr_active_pipeline.get_session_context(), aclu_column_name=None)


_neuron_replay_stats_df = HDF_Converter.prepare_neuron_indexed_dataframe_for_hdf(_neuron_replay_stats_df, active_context=curr_active_pipeline.get_session_context(), aclu_column_name=None)



# active_context = curr_active_pipeline.get_session_context()
# sess_specific_aclus = list(_neuron_replay_stats_df.index.to_numpy())
# session_ctxt_key:str = active_context.get_description(separator='|', subset_includelist=IdentifyingContext._get_session_context_keys())

# # Adds column columns=['neuron_uid', 'session_uid', 'aclu']
# _neuron_replay_stats_df['aclu'] = sess_specific_aclus # _neuron_replay_stats_df.index.to_numpy() # add explicit 'aclu' column from index
# _neuron_replay_stats_df['session_uid'] = session_ctxt_key # add fixed 'session_uid' column 
# _neuron_replay_stats_df['neuron_uid'] = [f"{session_ctxt_key}|{aclu}" for aclu in sess_specific_aclus]
_neuron_replay_stats_df



In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:

# _out = curr_active_pipeline.last_added_display_output
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
_out = curr_active_pipeline.display('_display_long_short_expected_v_observed_firing_rate', max_num_rows=10)
# a_fig = _out.figures[0]
# a_fig.show()
# .show()
# plt.show()


In [ ]:
expected_v_observed_result

In [ ]:
rate_remapping_df

In [ ]:
## Use the `long_short_decoding_analyses` global result to access `long_results_obj.active_filter_epochs`:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global


In [ ]:
# Ratemap
# ratemap = deepcopy(long_pf1D.ratemap)

ratemap: Ratemap = deepcopy(long_pf2D.ratemap)
ratemap

In [ ]:
# Plot long|short firing rate index:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']

# final_context = active_context.adding_context('display_fn', display_fn_name='display_long_short_laps')
# fig = _plot_long_short_firing_rate_indicies(x_frs_index, y_frs_index, final_context, debug_print=debug_print)

In [ ]:
long_short_fr_indicies_analysis_results

In [ ]:
h5_test_file_path = Path(f'output/test_long_short_fr_indicies_analysis_results.h5').resolve()
h5_test_file_path
long_short_fr_indicies_analysis_results.to_hdf(h5_test_file_path, key='long_short_fr_indicies_analysis_results')

# jonathan_firing_rate_analysis_result.is_hdf_serializable()

In [ ]:

h5_test_file_path = Path(f'output/test_long_short_decoding_analyses.h5').resolve()
h5_test_file_path
curr_long_short_decoding_analyses.to_hdf(h5_test_file_path, key='long_short_decoding_analyses')

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import LeaveOneOutDecodingResult, LeaveOneOutDecodingAnalysisResult, TimebinnedNeuronActivity, _convert_dict_to_hdf_attrs_fn

# curr_long_short_decoding_analyses.long_results_obj.new_result


In [ ]:

print(f'type(curr_long_short_decoding_analyses.long_results_obj): {type(curr_long_short_decoding_analyses.long_results_obj)}')
h5_test_file_path = Path(f'output/test_LeaveOneOutDecodingAnalysisResult.h5').resolve()
curr_long_short_decoding_analyses.long_results_obj.to_hdf(h5_test_file_path, key='long_results_obj')


In [ ]:
curr_long_short_decoding_analyses.short_results_obj.to_hdf(h5_test_file_path, key='short_results_obj')

In [ ]:


_obj = deepcopy(curr_long_short_decoding_analyses.long_results_obj.timebinned_neuron_info)

In [ ]:
# np.hstack((_obj.active_IDXs, _obj.inactive_IDXs))

[np.hstack((a_v, i_v)) for a_v, i_v in zip(_obj.active_IDXs, _obj.inactive_IDXs)]

In [ ]:
# TimebinnedNeuronActivity
h5_test_file_path = Path(f'output/test_TimebinnedNeuronActivity.h5').resolve()
curr_long_short_decoding_analyses.long_results_obj.timebinned_neuron_info.to_hdf(h5_test_file_path, key='timebinned_neuron_info')

In [ ]:
print(f'type(curr_long_short_decoding_analyses.long_results_obj.new_result): {type(curr_long_short_decoding_analyses.long_results_obj.new_result)}')
h5_test_file_path = Path(f'output/test_LeaveOneOutDecodingResult.h5').resolve()
curr_long_short_decoding_analyses.long_results_obj.new_result.to_hdf(h5_test_file_path, key='new_result', file_mode='r+')

In [ ]:
curr_long_short_decoding_analyses.short_results_obj

In [ ]:
jonathan_firing_rate_analysis_result.rdf.rdf


In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from neuropy.core.neurons import NeuronType

jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

h5_test_file_path = Path(f'output/test_jonathan_firing_rate_analysis_result.h5').resolve()
h5_test_file_path
jonathan_firing_rate_analysis_result.to_hdf(h5_test_file_path, key='jonathan_firing_rate_analysis_result')

# jonathan_firing_rate_analysis_result.is_hdf_serializable()

In [ ]:
aclu_to_idx = jonathan_firing_rate_analysis_result.rdf.aclu_to_idx
aclu_to_idx_df: pd.DataFrame = pd.DataFrame({'aclu': list(aclu_to_idx.keys()), 'fragile_linear_idx': list(aclu_to_idx.values())})

In [ ]:
# I wanna look at the suprise metrics again please.

# can align each of the sessions (across days) to the track transition point (the "Delta") as the zero.

# print_keys_if_possible("global_computation_results", curr_active_pipeline.global_computation_results, max_depth=2)

# computed_data: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters
# 		│   ├── jonathan_firing_rate_analysis: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters
# 		│   ├── long_short_fr_indicies_analysis: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters
# 		│   ├── long_short_leave_one_out_decoding_analysis: pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations.LeaveOneOutDecodingAnalysis
# 		│   ├── long_short_post_decoding: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters


print_keys_if_possible("global_computation_results", curr_active_pipeline.global_computation_results.computed_data, max_depth=3)

# jonathan_firing_rate_analysis_result.rdf

# long_short_leave_one_out_decoding_analysis

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding.expected_v_observed_result
# curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding

In [ ]:
curr_active_pipeline.global_computation_results.computed_data

In [ ]:
# Iterative partition
curr_active_pipeline.global_computation_results.computed_data

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding.expected_v_observed_result

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, InstantaneousFiringRatesDataframeAccessor, InstantaneousSpikeRateGroupsComputation, trackMembershipTypesEnum, trackExclusiveToMembershipTypeDict, trackExclusiveToMembershipTypeReverseDict

# ## Specify the output file:
# common_file_path = Path('output/test_across_session_scatter_plot_new.h5')
# print(f'common_file_path: {common_file_path}')
# InstantaneousFiringRatesDataframeAccessor.add_results_to_inst_fr_results_table(curr_active_pipeline, common_file_path, file_mode='a')

In [ ]:
import seaborn as sns
from pyphocorehelpers.indexing_helpers import partition, safe_pandas_get_group
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor, AcrossSessionsVisualizations
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import SingleBarResult

common_file_path = Path('output/active_across_session_scatter_plot_results.h5')
print(f'common_file_path: {common_file_path}')

# InstantaneousSpikeRateGroupsComputation, : pd.DataFrame
_shell_obj, loaded_result_df = InstantaneousFiringRatesDataframeAccessor.load_and_prepare_for_plot(common_file_path)
# Perform the actual plotting:
AcrossSessionsVisualizations.across_sessions_bar_graphs(_shell_obj, num_sessions=13, save_figure=False, enable_tiny_point_labels=False, enable_hover_labels=True)

In [ ]:
from datetime import datetime, timedelta

maximum_timedelta: timedelta = timedelta(1, 0, 0) # 1 Day maximum time
delta_since_last_compute: timedelta = curr_active_pipeline.get_time_since_last_computation()
print(f'delta_since_last_compute: {delta_since_last_compute}')
needs_recompute: bool = delta_since_last_compute > maximum_timedelta
print(f'\tneeds_recompute: {needs_recompute}')

In [ ]:
## Serialization of preprocessing parameters test
file_path = 'output/preprocessing_parameters.h5'

with h5py.File(file_path, "w") as f:
	preprocessing_group = f.create_group("preprocessing_parameters")

	# Serialize epoch_estimation_parameters
	epoch_estimation_group = preprocessing_group.create_group("epoch_estimation_parameters")

	laps_group = epoch_estimation_group.create_group("laps")
	laps_group.attrs["N"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["laps"]["N"]
	laps_group.attrs["should_backup_extant_laps_obj"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["laps"]["should_backup_extant_laps_obj"]

	PBEs_group = epoch_estimation_group.create_group("PBEs")
	PBEs_group.attrs["thresh"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["thresh"]
	PBEs_group.attrs["min_dur"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["min_dur"]
	PBEs_group.attrs["merge_dur"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["merge_dur"]
	PBEs_group.attrs["max_dur"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["max_dur"]

	replays_group = epoch_estimation_group.create_group("replays")
	replay_data = preprocessing_parameters_dict["epoch_estimation_parameters"]["replays"]
	# replay_dataframe = pd.DataFrame(replay_data)
	# replay_data.to_hdf(f, "/preprocessing_parameters/epoch_estimation_parameters/replays")
	## TODO: add the data here using Epoch's .to_hdf
	
	# Check for "None" values before setting attributes
	def check_and_set(key, value):
		if value is not None:
			replays_group.attrs[key] = value

	# Set attributes if not "None", otherwise skip writing
	check_and_set("min_epoch_included_duration", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("min_epoch_included_duration"))
	check_and_set("max_epoch_included_duration", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("max_epoch_included_duration"))
	check_and_set("maximum_speed_thresh", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("maximum_speed_thresh"))
	check_and_set("min_inclusion_fr_active_thresh", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("min_inclusion_fr_active_thresh"))
	check_and_set("min_num_unique_aclu_inclusions", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("min_num_unique_aclu_inclusions"))

In [ ]:
enable_default_neptune_plots = False
# enable_default_neptune_plots = True

## To file only:
with matplotlib_file_only():
	# Perform non-interactive Matplotlib operations with 'AGG' backend
	main_complete_figure_generations(curr_active_pipeline, enable_default_neptune_plots=enable_default_neptune_plots, save_figures_only=True, save_figure=True)

## Clear the Programmatically open figures:
plt.close('all') # this takes care of the matplotlib-backed figures.
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_out_figures = main_complete_figure_generations(curr_active_pipeline, enable_default_neptune_plots=False, save_figures_only=False, save_figure=True)

# Common Display Setup

In [ ]:
import matplotlib
# configure backend here
matplotlib.use('Qt5Agg')
# backend_qt5agg
# %matplotlib qt5
# %matplotlib qt
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars

# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

In [ ]:
# import pylustrator # customization of figures
import matplotlib
# configure backend here
matplotlib.use('Qt5Agg')
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
import pylustrator # call `pylustrator.start()` before creating your first figure in code.

In [ ]:
pylustrator.start()

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

In [ ]:
# Plot The laps:
from neuropy.utils.matplotlib_helpers import plot_position_curves_figure
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_laps_2d
pylustrator.start()
fig, out_axes_list = plot_laps_2d(global_session, legacy_plotting_mode=False, include_velocity=False, include_accel=False, figsize=(24, 10))
out_axes_list[0].set_title('Estimated Laps')
fig.canvas.manager.set_window_title('Estimated Laps')
ax = out_axes_list[0]
ax.set_xlim((1036.242685185441, 1441.769668184419))

# Hide y-axis ticklabels
plt.tick_params(axis='y', which='both', labelleft=False)

In [ ]:
fig = plt.gcf()
ax = fig.axes[0]
ax
ax.get_xlim()


laps_example_region_xlims = (1036.242685185441, 1441.769668184419)

fig, out_axes_list = plot_position_curves_figure(global_session.position, include_velocity=False, include_accel=False, figsize=(24, 10))

_out1 = curr_active_pipeline.display('_display_long_short_laps', include_velocity=False, include_accel=False)


In [ ]:
curr_active_pipeline.plot._display_decoder_result()

# 2023-09-07 - Track Graphics Testing

In [ ]:
from pyphocorehelpers.geometry_helpers import BoundsRect, point_tuple_mid_point, map_value
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import LinearTrackDimensions
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import add_vertical_track_bounds_lines, add_track_shapes, test_LinearTrackDimensions_2D_pyqtgraph
from neuropy.utils.mathutil import contiguous_regions, threshPeriods, compute_grid_bin_bounds, map_value
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import test_LinearTrackDimensions_2D_Matplotlib
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import LinearTrackInstance

grid_bin_bounds = BoundsRect.init_from_grid_bin_bounds(global_pf2D.config.grid_bin_bounds)
display(grid_bin_bounds)

# long_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(grid_bin_bounds)
# short_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(grid_bin_bounds)

long_track_dims = LinearTrackDimensions(track_length=170.0)
short_track_dims = LinearTrackDimensions(track_length=100.0)

common_1D_platform_height = 0.25
common_1D_track_height = 0.1
long_track_dims.minor_axis_platform_side_width = common_1D_platform_height
long_track_dims.track_width = common_1D_track_height # (short_track_dims.minor_axis_platform_side_width

short_track_dims.minor_axis_platform_side_width = common_1D_platform_height
short_track_dims.track_width = common_1D_track_height # (short_track_dims.minor_axis_platform_side_width

# instances:
long_track = LinearTrackInstance(long_track_dims, grid_bin_bounds=grid_bin_bounds)
short_track = LinearTrackInstance(short_track_dims, grid_bin_bounds=grid_bin_bounds)

print(long_track_dims)
print(short_track_dims)

In [ ]:
import napari

# global_pf1D_dt.config.grid_bin_bounds


def napari_add_grid_bin_bounds_rect(viewer: napari.viewer.Viewer, grid_bin_bounds):
	""" adds the animal's position as a track and a point layer to the napari viewer.


	Usage:

	from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_add_animal_position
	napari_add_animal_position(viewer=viewer, position_df=global_pf1D_dt.all_time_filtered_pos_df[['t','x','binned_x']], time_intervals)


	"""
	# rect = np.array([[0, 0], [3, 1]])
	grid_bin_bounds_rect = grid_bin_bounds
	viewer.add_shapes(grid_bin_bounds_rect, shape_type='grid_bin_bounds_rect', edge_width=0.1)
    
napari_add_grid_bin_bounds_rect(viewer, grid_bin_bounds)

In [ ]:
## For napari need to map onto xbins axis:
#TODO: Not working, did manual rect drawing below instead.

assert len(viewer.dims.range) == 3
t_range_tuple, neuron_id_range_tuple, xbin_range_tuple = viewer.dims.range
# ((0.0, 3309.0, 1.0),
#  (-0.2906298631133275, 85.0, 1.0),
#  (-0.24635407377607876, 108.0, 1.0))
xbin_min, xbin_max = (xbin_range_tuple[0], (xbin_range_tuple[1]-xbin_range_tuple[2])) # the max range isn't included, so we need to subtract off the step to get the real max
xbin_width = (xbin_max - xbin_min) # total width in number of xbins
x_to_xbins_multiplier: float = xbin_width/grid_bin_bounds.size[0] # multiply a number in normal coordinates to get the equivalent number of xbins
x_to_xbins_multiplier

map_track_coords_to_xbins_space = lambda v: map_value(v, (grid_bin_bounds.xmin, grid_bin_bounds.xmax), (xbin_min, xbin_max)) # same map
# map_track_coords_to_ybins_space = lambda v: map_value(v, (grid_bin_bounds.ymin, grid_bin_bounds.ymax), (ybin_min, ybin_max)) # same map

bin_space_grid_bin_bounds = deepcopy(grid_bin_bounds)
bin_space_grid_bin_bounds.xmin = map_track_coords_to_xbins_space(grid_bin_bounds.xmin)
bin_space_grid_bin_bounds.xmax = map_track_coords_to_xbins_space(grid_bin_bounds.xmax)

bin_space_grid_bin_bounds.ymin = 0.0
bin_space_grid_bin_bounds.ymax = 5.0

bin_space_grid_bin_bounds

# map_track_coords_to_xbins_space(grid_bin_bounds.xmin), map_track_coords_to_xbins_space(grid_bin_bounds.xmax)
# bin_space_long_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(bin_space_grid_bin_bounds)
# bin_space_short_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(bin_space_grid_bin_bounds)

# common_1D_platform_height = 0.25
# common_1D_track_height = 0.1
# bin_space_long_track_dims.minor_axis_platform_side_width = common_1D_platform_height
# bin_space_long_track_dims.track_width = common_1D_track_height # (short_track_dims.minor_axis_platform_side_width

# bin_space_short_track_dims.minor_axis_platform_side_width = common_1D_platform_height
# bin_space_short_track_dims.track_width = common_1D_track_height # (short_track_dims.minor_axis_platform_side_width

bin_space_long_track_dims = deepcopy(long_track_dims)
bin_space_short_track_dims = deepcopy(short_track_dims)

bin_space_long_track_dims.axis_scale_factors = (x_to_xbins_multiplier, 1.0)
bin_space_short_track_dims.axis_scale_factors = (x_to_xbins_multiplier, 1.0)

bin_space_long_track = LinearTrackInstance(bin_space_long_track_dims, grid_bin_bounds=bin_space_grid_bin_bounds)
bin_space_short_track = LinearTrackInstance(bin_space_short_track_dims, grid_bin_bounds=bin_space_grid_bin_bounds)

bin_space_short_track

t_range_tuple, neuron_id_range_tuple, xbin_range_tuple = viewer.dims.range
t_range_tuple


bin_space_grid_bin_bounds
half_xbin_width = float(xbin_width)/2.0
half_xbin_width
bin_space_short_track.track_dimensions.scaled_total_length
# vertices are top-left and bottom-right corners
# [(x, 0.0, w, h) for x, y, w, h, *_unused in bin_space_short_track.rects]
# short_track_rectangle_data = [BoundsRect.init_from_x_y_w_h_tuple((x+np.abs(x), y, w, h)).corner_points for x, y, w, h, *_unused in bin_space_short_track.rects]
short_track_rectangle_data = [np.hstack((np.zeros((4,1)), BoundsRect.init_from_x_y_w_h_tuple((x+half_xbin_width, y, w, h)).corner_points))[:, :][:, [0, 2, 1]] for x, y, w, h, *_unused in bin_space_short_track.rects]
long_track_rectangle_data = np.vstack([np.hstack((np.zeros((4,1)), BoundsRect.init_from_x_y_w_h_tuple((x+half_xbin_width, y, w, h)).corner_points))[:, [0, 2, 1]] for x, y, w, h, *_unused in bin_space_long_track.rects])

# short_track_rectangle_data = np.vstack(short_track_rectangle_data)
short_track_rectangle_data

# array([[0, 2.375, 35], # bottom-left
    #    [0, 2.625, 35], # bottom-right
    #    [0, 2.375, 46.112], # top-left
    #    [0, 2.625, 46.112], # top-right

# [0,2,3,1]

In [ ]:
import itertools

# sync_point_y = -0.750314
sync_point_y = 0.0 # the bottom (y-axis) value of the track
track_platform_height_y = -5.19389 # the top coordinate (y-axis) value of the platforms

## Time indicies for tracks:
time_index = 0
long_short_change_time_index = 2097 # APPROXIMATE
t_range_tuple, neuron_id_range_tuple, xbin_range_tuple = viewer.dims.range
long_time_indicies = np.arange(t_range_tuple[0], long_short_change_time_index)
short_time_indicies = np.arange(long_short_change_time_index, (t_range_tuple[1]-t_range_tuple[2]))

### Long Track Shape Layer:
# Where the platform and the track connect:
track_platform_connect_x_min = 14.157
track_platform_connect_x_max = 91.6898
long_track_shapes_data = list(itertools.chain.from_iterable([
[
np.array([[time_index, track_platform_height_y, track_platform_connect_x_max],
        [time_index, track_platform_height_y, 106.093],
        [time_index, sync_point_y, 106.093],
        [time_index, sync_point_y, track_platform_connect_x_max]]),
 np.array([[time_index, -3.66162, 14.0037],
        [time_index, -3.66162, track_platform_connect_x_max],
        [time_index, sync_point_y, track_platform_connect_x_max],
        [time_index, sync_point_y, 14.0037]]),
 np.array([[time_index, track_platform_height_y, -0.246354],
        [time_index, track_platform_height_y, track_platform_connect_x_min],
        [time_index, sync_point_y, track_platform_connect_x_min],
        [time_index, sync_point_y, -0.246354]])
]
for time_index in long_time_indicies]))
long_track_shapes_layer = viewer.add_shapes(long_track_shapes_data, shape_type='rectangle', name='long_track', edge_width=0, face_color='#aa0000ff')
long_track_shapes_layer.editable = False


### Short Track Shape Layer:
# Where the platform and the track connect:
track_platform_connect_x_min = 28.2539
track_platform_connect_x_max = 73.7623

short_track_shapes_data =  list(itertools.chain.from_iterable([
[
np.array([[time_index, track_platform_height_y, track_platform_connect_x_max],
        [time_index, track_platform_height_y, 88.1655],
        [time_index, sync_point_y, 88.1655],
        [time_index, sync_point_y, track_platform_connect_x_max]]),
np.array([[time_index, -3.66162, 28.1006],
        [time_index, -3.66162, track_platform_connect_x_max],
        [time_index, sync_point_y, track_platform_connect_x_max],
        [time_index, sync_point_y, 28.1006]]),
np.array([[time_index, track_platform_height_y, 13.8505],
        [time_index, track_platform_height_y, track_platform_connect_x_min],
        [time_index, sync_point_y, track_platform_connect_x_min],
        [time_index, sync_point_y, 13.8505]])
]
for time_index in short_time_indicies]))

short_track_shapes_layer = viewer.add_shapes(short_track_shapes_data, shape_type='rectangle', name='short_track', edge_width=0, face_color='royalblue')
short_track_shapes_layer.editable = False

In [ ]:
long_track_shapes_layer.data

In [ ]:
shapes_layer = viewer.layers['short_track_rectangle_data']
shapes_layer.data





# Test classifying various x-positions as belonging to outside the outside_maze, the track_endcaps, or the track_body

In [ ]:
long_rects_outputs, short_rects_outputs = add_track_shapes(grid_bin_bounds, ax=None)

In [ ]:
# long_offset=(150.0, 0.5)
# short_offset=(150.0, -0.5)

long_offset=(grid_bin_bounds.center_point[0], 0.5)
short_offset=(grid_bin_bounds.center_point[0], -0.5)

# # Create a second y-axis sharing the same x-axis as ax
# ax2 = ax.twinx()
# # ax2.plot(x, y2, 'b-')
# ax2.set_ylabel('Track data', color='b')
# # Set the adjustable attribute to 'datalim'
# ax.set_adjustable('datalim')
# ax2.set_adjustable('datalim')
# long_track_dims.plot_rects(ax2, offset=long_offset)
# short_track_dims.plot_rects(ax2, offset=short_offset)

combined_item, rect_items, rects = long_track_dims.plot_rects(ax, offset=long_offset)
short_track_dims.plot_rects(ax, offset=short_offset)

In [ ]:
fig, ax1, ax2 = test_LinearTrackDimensions_2D_Matplotlib(long_track_dims, short_track_dims, long_offset=(150.0, 0.5), short_offset=(150.0, -0.5))

In [ ]:

# Find center from `grid_bin_bounds`
# long_pf2D.bin_info
grid_bin_bounds = deepcopy(long_pf2D.config.grid_bin_bounds)

x_diff = (grid_bin_bounds[0][1] - grid_bin_bounds[0][0])
y_diff = (grid_bin_bounds[1][1] - grid_bin_bounds[1][0])
print(f'x_diff: {x_diff}, y_diff: {y_diff}')

# Find the x, y midpoints:
x_midpoint = grid_bin_bounds[0][0] + (x_diff/2.0)
y_midpoint = grid_bin_bounds[1][0] + (y_diff/2.0)
print(f'x_midpoint: {x_midpoint}, y_midpoint: {y_midpoint}')


In [ ]:
grid_bin_bounds_center_point = (point_tuple_mid_point(grid_bin_bounds[0]), point_tuple_mid_point(grid_bin_bounds[1])) # (145.43, 140.61)
x_midpoint, y_midpoint = grid_bin_bounds_center_point

In [ ]:

# long_track_dims.total_length # 214.0
# short_track_dims.total_length # 144.0
# zero_alignment_point = self.total_length/2.0
long_notable_x_positions, _long_notable_y_positions = long_track_dims._build_component_notable_positions(offset_point=(x_midpoint, y_midpoint))
short_notable_x_positions, _short_notable_y_positions = short_track_dims._build_component_notable_positions(offset_point=(x_midpoint, y_midpoint))

In [ ]:
# def _add_vertical_track_bounds_lines(grid_bin_bounds, ax=None):
# 	""" 
# 	Captures: long_notable_x_positions, short_notable_x_positions
	
# 	Usage:
# 		grid_bin_bounds = deepcopy(long_pf2D.config.grid_bin_bounds)
# 		long_track_line_collection, short_track_line_collection = _add_vertical_track_bounds_lines(grid_bin_bounds=grid_bin_bounds, ax=ax)

# 	"""
# 	long_track_dims = LinearTrackDimensions(track_length=170.0)
# 	short_track_dims = LinearTrackDimensions(track_length=100.0)

# 	# Find center from `grid_bin_bounds` using `point_tuple_mid_point`
# 	x_midpoint, y_midpoint = (point_tuple_mid_point(grid_bin_bounds[0]), point_tuple_mid_point(grid_bin_bounds[1])) # grid_bin_bounds_center_point: (145.43, 140.61)

# 	long_notable_x_positions, _long_notable_y_positions = long_track_dims._build_component_notable_positions(offset_point=(x_midpoint, y_midpoint))
# 	short_notable_x_positions, _short_notable_y_positions = short_track_dims._build_component_notable_positions(offset_point=(x_midpoint, y_midpoint))

# 	# Omit the midpoint
# 	long_notable_x_platform_positions = long_notable_x_positions[[0,1,3,4]]
# 	short_notable_x_platform_positions = short_notable_x_positions[[0,1,3,4]]

# 	## Adds to current axes:
# 	if ax is None:
# 		fig = plt.gcf()
# 		axs = fig.get_axes()
# 		ax = axs[0]
# 	long_track_line_collection: matplotlib.collections.LineCollection = plt.vlines(long_notable_x_platform_positions, label='long_track_x_pos_lines', ymin=ax.get_ybound()[0], ymax=ax.get_ybound()[1], colors='#0000FFAA', linestyles='dashed') # matplotlib.collections.LineCollection
# 	short_track_line_collection: matplotlib.collections.LineCollection = plt.vlines(short_notable_x_platform_positions, label='short_track_x_pos_lines', ymin=ax.get_ybound()[0], ymax=ax.get_ybound()[1], colors='#FF0000AA', linestyles='dashed') # matplotlib.collections.LineCollection
# 	return long_track_line_collection, short_track_line_collection

grid_bin_bounds = deepcopy(long_pf2D.config.grid_bin_bounds)
long_track_line_collection, short_track_line_collection = add_vertical_track_bounds_lines(grid_bin_bounds=grid_bin_bounds, ax=None)

In [ ]:
rect_items, rects = long_track_dims.plot_rects(axs[0]) # , offset=(x, y) # , offset=(0.1, 100.0)


In [ ]:
long_track_line_collection.remove()

In [ ]:
short_track_line_collection.remove()

In [ ]:
long_track_dims.plot_rects(ax, offset=)


In [ ]:
fig = plt.gcf()
axs = fig.get_axes()
ax = axs[0]

In [ ]:
long_track_line_collection.remove()
short_track_line_collection.remove()

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_1D_placecell_validation

_out = plot_1D_placecell_validation(long_pf1D, 0)

axes = _out[1]
ax = axes[0]
# ax = axes[1]
ax

In [ ]:
long_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(grid_bin_bounds=grid_bin_bounds, debug_print=True)

fig, axs = plt.subplots(2, 1)

rect_items, rects = long_track_dims.plot_rects(axs[0]) # , offset=(x, y) # , offset=(0.1, 100.0)


In [ ]:

# long_track_dims.plot_rects(ax)

In [ ]:
_out = curr_active_pipeline.display('_display_grid_bin_bounds_validation', curr_active_pipeline.get_session_context(), defer_render=False, save_figure=True)
fig = _out.figures[0]
ax = _out.axes[0]

In [ ]:
# Extract the bounding box coordinates and dimensions
bbox = ax.bbox
x, y, width, height = bbox.x0, bbox.y0, bbox.width, bbox.height
x, y, width, height

In [ ]:
ax_empty = ax

In [ ]:
rect_items, rects = long_track_dims.plot_rects(ax_empty) # , offset=(x, y) # , offset=(0.1, 100.0)


In [ ]:
(320.0, 983.7668560462606, 901.8181818181818, 91.99628790747863)
active_track_dims = LinearTrackDimensions(width=901.8181818181818)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.transforms import Bbox

# Step 2: Define a simple vector graphic (an arrow in this case)
def draw_arrow(ax, x, y, width, height):
    arrow = patches.FancyArrow(x, y, width, height, width=0.2*height, color="red")
    ax.add_patch(arrow)

# Step 3: Scale and draw the arrow within a bounding box
def draw_scaled_arrow_in_bbox(ax, bbox):
    # Extract the bounding box coordinates and dimensions
    x, y, width, height = bbox.x0, bbox.y0, bbox.width, bbox.height

    # Draw the bounding box (for visualization)
    rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='black', facecolor='none')
    ax.add_patch(rect)

    # Draw the scaled arrow within the bounding box
    draw_arrow(ax, x, y, width, height)

# Main code
fig, ax = plt.subplots()
bbox = Bbox.from_bounds(0.2, 0.2, 0.6, 0.6)  # Define bounding box with [x, y, width, height]

draw_scaled_arrow_in_bbox(ax, bbox)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
plt.show()


In [ ]:
plt.draw()

In [ ]:
rect_items[0]

In [ ]:
rects

# 2023-09-21 - Continuous Surprise Figure - How do I display it?

In [ ]:
# '_perform_time_dependent_pf_sequential_surprise_computation'
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_surprise_difference_plot, plot_long_short, plot_long_short_any_values
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import fig_surprise_results

from neuropy.utils.matplotlib_helpers import draw_epoch_regions
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import LongShortDisplayConfigManager
long_short_display_config_manager = LongShortDisplayConfigManager()
long_epoch_config = long_short_display_config_manager.long_epoch_config.as_matplotlib_kwargs()
short_epoch_config = long_short_display_config_manager.short_epoch_config.as_matplotlib_kwargs()

graphics_outputs_list = fig_surprise_results(curr_active_pipeline)


### FOUND! 2023-09-21 - Found code that generated surprise plots, strangely not stored anywhere!

In [ ]:
# note: nSnapshots == n_post_update_times
active_relative_entropy_results = active_extended_stats['pf_dt_sequential_surprise']
post_update_times = active_relative_entropy_results['post_update_times'] # (4123,) = (nSnapshots,)
snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
time_intervals = active_relative_entropy_results['time_intervals']
long_short_rel_entr_curves_frames = active_relative_entropy_results['long_short_rel_entr_curves_frames'] # (4123, 108, 63) = (nSnapshots, n_neurons, n_xbins)
short_long_rel_entr_curves_frames = active_relative_entropy_results['short_long_rel_entr_curves_frames'] # (4123, 108, 63) = (nSnapshots, n_neurons, n_xbins)
flat_relative_entropy_results = active_relative_entropy_results['flat_relative_entropy_results'] # (4123, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_results = active_relative_entropy_results['flat_jensen_shannon_distance_results'] # (4123, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_across_all_positions = np.sum(np.abs(flat_jensen_shannon_distance_results), axis=1) # sum across all position bins # (4123,) - (nSnapshots)
flat_surprise_across_all_positions = np.sum(np.abs(flat_relative_entropy_results), axis=1) # sum across all position bins # (4123,) - (nSnapshots)
nSnapshots, n_neurons, n_xbins = np.shape(long_short_rel_entr_curves_frames)
print(f'nSnapshots: {nSnapshots}, n_neurons: {n_neurons}, n_xbins: {n_xbins}') # nSnapshots: 3308, n_neurons: 85, n_xbins: 107

In [ ]:
# post_update_times.shape
# np.shape(post_update_times)[0]



In [ ]:
np.shape(flat_jensen_shannon_distance_results)

In [ ]:
np.shape(flat_relative_entropy_results)

In [ ]:
np.shape(flat_jensen_shannon_distance_across_all_positions)

In [ ]:
np.shape(long_short_rel_entr_curves_frames)

In [ ]:
## NOTE: these results are NOT the same as the ones calculated in compute_snapshot_relative_entropy_surprise_differences.compute_surprise_relative_entropy_divergence, and I'm not quite sure why:
# Jensen-Shannon distance is an average of KL divergence:
from scipy.special import rel_entr
mixture_distribution = 0.5 * (long_short_rel_entr_curves_frames + short_long_rel_entr_curves_frames)
print(f'mixture_distribution.shape: {np.shape(mixture_distribution)}') # (nSnapshots, n_neurons, n_xbins)
# jensen_shannon_distance = 0.5 * (sum(rel_entr(mixture_distribution, long_short_rel_entr_curves_frames)) + sum(rel_entr(mixture_distribution, short_long_rel_entr_curves_frames))) # is this right? I'm confused by sum(...) # (n_neurons, n_xbins)
jensen_shannon_distance = 0.5 * (np.sum(rel_entr(mixture_distribution, long_short_rel_entr_curves_frames), axis=1) + np.sum(rel_entr(mixture_distribution, short_long_rel_entr_curves_frames), axis=1)) # alt version: (nSnapshots, n_xbins)
print(f'jensen_shannon_distance.shape: {np.shape(jensen_shannon_distance)}') # (n_neurons, n_xbins)

In [ ]:
np.isclose(flat_jensen_shannon_distance_results, jensen_shannon_distance).all()


In [ ]:
viewer.add_image(flat_jensen_shannon_distance_results)

In [ ]:
from silx.gui.plot import Plot2D

plot = Plot2D()  # Create the plot widget
plot.addImage(flat_jensen_shannon_distance_results, legend='flat_jensen_shannon_distance_results')  # Plot the 2D data set with default colormap
plot.setGraphTitle('flat_jensen_shannon_distance_results')
plot.getXAxis().setLabel('Position Bin')
plot.getYAxis().setLabel('Snapshot Timebin')
plot.show()  # Make the plot widget visible


In [ ]:
plot = Plot2D()  # Create the plot widget
plot.addImage(jensen_shannon_distance, legend='jensen_shannon_distance')  # Plot the 2D data set with default colormap
plot.setGraphTitle('jensen_shannon_distance')
plot.getXAxis().setLabel('Position Bin')
plot.getYAxis().setLabel('Snapshot Timebin')
plot.show()  # Make the plot widget visible

In [ ]:

plot = Plot2D()  # Create the plot widget
plot.addImage(long_short_rel_entr_curves_frames, legend='image')  # Plot the 2D data set with default colormap
plot.setGraphTitle('long_short_rel_entr_curves_frames')
# plot.getXAxis().setLabel('Position Bin')
# plot.getYAxis().setLabel('Snapshot Timebin')
plot.show()  # Make the plot widget visible


In [ ]:
from silx.gui.plot.StackView import StackViewMainWindow

# synthetic data, stack of 100 images of size 200x300
# mystack = np.fromfunction(
#     lambda i, j, k: np.sin(i/15.) + np.cos(j/4.) + 2 * np.sin(k/6.),
#     (100, 200, 300)
# )

sv = StackViewMainWindow()
sv.setColormap("jet", autoscale=True)
sv.setStack(long_short_rel_entr_curves_frames)
# (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
sv.setLabels(["post_update_time", "aclu", "Position (xbin)"])
sv.show()

In [ ]:


heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(flat_jensen_shannon_distance_results, show_yticks=False, title=f"flat_jensen_shannon_distance_results", defer_show=False)

In [ ]:
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(flat_jensen_shannon_distance_across_all_positions, show_yticks=False, title=f"flat_jensen_shannon_distance_across_all_positions")

In [ ]:
from neuropy.utils.matplotlib_helpers import draw_epoch_regions
from neuropy.core.epoch import Epoch


fig, ax = plt.subplots()
ax.plot(post_update_times, flat_surprise_across_all_positions)
ax.set_ylabel('Relative Entropy across all positions')
ax.set_xlabel('t (seconds)')
epochs_collection, epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.epochs, ax, facecolor=('r','cyan'), alpha=0.1, edgecolors=None, labels_kwargs={'y_offset': -0.05, 'size': 14}, defer_render=True, debug_print=False)
laps_epochs_collection, laps_epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.laps.as_epoch_obj(), ax, facecolor='r', edgecolors='black', labels_kwargs={'y_offset': -16.0, 'size':8}, defer_render=True, debug_print=False)
# replays_epochs_collection, replays_epoch_labels = draw_epoch_regions(active_filter_epoch_obj, ax, facecolor='orange', edgecolors=None, labels_kwargs=None, defer_render=False, debug_print=False)
fig.suptitle('flat_surprise_across_all_positions')
fig.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(post_update_times, flat_jensen_shannon_distance_across_all_positions, label='JS_Distance')
ax.set_ylabel('J-S Distance across all positions')
ax.set_xlabel('t (seconds)')
epochs_collection, epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.epochs, ax, facecolor=('red','cyan'), alpha=0.1, edgecolors=None, labels_kwargs={'y_offset': -0.05, 'size': 14}, defer_render=True, debug_print=False)
laps_epochs_collection, laps_epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.laps.as_epoch_obj(), ax, facecolor='red', edgecolors='black', labels_kwargs={'y_offset': -16.0, 'size':8}, defer_render=True, debug_print=False)
# replays_epochs_collection, replays_epoch_labels = draw_epoch_regions(active_filter_epoch_obj, ax, facecolor='orange', edgecolors=None, labels_kwargs=None, defer_render=False, debug_print=False)
fig.suptitle('flat_jensen_shannon_distance_across_all_positions')
fig.show()

In [ ]:
# Show basic relative entropy vs. time plot:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(post_update_times, flat_relative_entropy_results)
ax.set_ylabel('Relative Entropy')
ax.set_xlabel('t (seconds)')
epochs_collection, epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.epochs, ax, facecolor=('red','cyan'), alpha=0.1, edgecolors=None, labels_kwargs={'y_offset': -0.05, 'size': 14}, defer_render=False, debug_print=False)
fig.show()

In [ ]:
win, plots = plot_long_short_surprise_difference_plot(curr_active_pipeline, long_results_obj, short_results_obj, long_epoch_name, short_epoch_name)

In [ ]:
win, (ax_long, ax_short), legend = plot_long_short(long_results_obj, short_results_obj)

In [ ]:
x_fn = lambda a_results_obj: a_results_obj.all_epochs_decoded_epoch_time_bins_mean[:,0]
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_scrambled_pf_surprises_mean
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
y_fn = lambda a_results_obj: a_results_obj.all_epochs_computed_one_left_out_posterior_to_pf_surprises

# (time_bins, neurons), (epochs, neurons), (epochs)
# all_epochs_computed_one_left_out_posterior_to_pf_surprises, all_epochs_computed_cell_one_left_out_posterior_to_pf_surprises_mean, all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
win, plots_tuple, legend = plot_long_short_any_values(long_results_obj, short_results_obj, x=x_fn, y=y_fn) #  limit_aclus=[4]

In [ ]:
##TODO 2023-09-21 16:19: - [ ] Does not work due to QCode issue

from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.MultiContextComparingDisplayFunctions import _context_nested_docks


# active_display_output = {}
# active_identifying_filtered_session_ctx = global_epoch_context
# display_output = active_display_output | curr_active_pipeline.display('_display_context_nested_docks', active_identifying_filtered_session_ctx, enable_gui=False, debug_print=False) # returns {'master_dock_win': master_dock_win, 'app': app, 'out_items': out_items}
# master_dock_win = display_output['master_dock_win']
# app = display_output['app']
# out_items = display_output['out_items']

include_includelist = curr_active_pipeline.active_completed_computation_result_names # ['maze', 'sprinkle']

out_items = {}
master_dock_win, app, out_items = _context_nested_docks(curr_active_pipeline, active_config_names=include_includelist, **{'enable_gui': True, 'debug_print': False})
master_dock_win

In [ ]:
neuron_replay_stats_df: pd.DataFrame = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
short_pf_peak_x = neuron_replay_stats_df.short_pf_peak_x.values
long_pf_peak_x = neuron_replay_stats_df.long_pf_peak_x.values

# long_pf_peak_x_maze_classification = [classify_x_position(test_x, long_rects) for test_x in long_pf_peak_x.values]
# short_pf_peak_x_maze_classification = [classify_x_position(test_x, short_rects) for test_x in short_pf_peak_x.values]
# [classify_x_position(test_x, short_rects) for test_x in long_pf_peak_x.values]

long_track.is_on_maze(long_pf_peak_x)
# long_track.is_on_endcap(long_pf_peak_x)

## NOW TODO 2023-09-20 22:33: - [ ] Need to fix tests and determing why some peaks are falling outside the bounds (as assessed by the is_on_maze function to see if implementation is valid

In [ ]:

# assert short_track.is_on_maze(short_pf_peak_x)[np.logical_not(np.isnan(short_pf_peak_x))].all(), f"all valid short peaks should be located on the short track" # should all be true yeah?
assert long_track.is_on_maze(short_pf_peak_x)[np.logical_not(np.isnan(short_pf_peak_x))].all(), f"all valid short peaks should be located on the long track"
assert long_track.is_on_maze(long_pf_peak_x)[np.logical_not(np.isnan(long_pf_peak_x))].all(), f"all valid long peaks should be located on the long track. {long_track.is_on_maze(long_pf_peak_x)[np.logical_not(np.isnan(long_pf_peak_x))]}" # not working
assert short_track.is_on_maze(short_pf_peak_x)[np.logical_not(np.isnan(short_pf_peak_x))].all(), f"all valid short peaks should be located on the short track. {short_track.is_on_maze(short_pf_peak_x)[np.logical_not(np.isnan(short_pf_peak_x))]}" # not working


In [ ]:
long_track.is_on_endcap(long_pf_peak_x)

## Debug Plots for `LinearTrackDimensions` and `LinearTrackInstance`

In [ ]:
app, w, cw, (long_track_dims, long_rect_items, long_rects), (short_track_dims, short_rect_items, short_rects) = test_LinearTrackDimensions_2D_pyqtgraph(long_track_dims, short_track_dims)


In [ ]:
fig = plt.gcf()
axs = fig.get_axes()
ax = axs[0]
ax

In [ ]:
import matplotlib.gridspec as gridspec

## Adds a new subplot to an existing (fig, ax) without requiring modifications in the original code!

# Get the current gridspec from ax
gs = ax.get_subplotspec().get_gridspec()

# Create a new gridspec with an additional column
gs_new = gridspec.GridSpec(1, 2, width_ratios=[1, 0.5]) # new column is half the width of the current one

# Reposition the existing ax using the new gridspec
ax.set_position(gs_new[0, 0].get_position(fig))

# Add a new subplot in the new column
ax2 = fig.add_subplot(gs_new[0, 1])


ax2.plot(np.cos(np.linspace(0, 10, 100)))
# ax2.cla()
# long_track_dims.plot_rects(ax2)

# long_track_dims.plot_rects(ax2, offset=(0.1, 0.0))

plt.tight_layout()
plt.show()


In [ ]:
_out2 = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', included_unit_neuron_IDs=[2], n_max_plot_rows=2, save_figure=False)


In [ ]:
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import fig_example_handpicked_pho_jonathan_active_set_cells

_LxC_out, _SxC_out = fig_example_handpicked_pho_jonathan_active_set_cells(curr_active_pipeline, save_figure=True, included_LxC_example_neuron_IDs=[4, 58], included_SxC_example_neuron_IDs=[2])

In [ ]:
# from neuropy.analyses.placefields import PfnDMixin, plotRaw_v_time
from neuropy.analyses.placefields import PfnDMixin

curr_active_pipeline.reload_default_display_functions()

_out3 = long_pf1D.plotRaw_v_time(0, should_include_trajectory=True, should_include_spikes=False, should_include_labels=True, use_filtered_positions=True, use_pandas_plotting=False)

In [ ]:
_out4 = long_pf1D.plotRaw_v_time(0, should_include_trajectory=True, should_include_spikes=False, should_include_labels=True, use_filtered_positions=False, use_pandas_plotting=False)
# _out4 = long_pf1D.plotRaw_v_time(0, should_include_trajectory=True, should_include_spikes=False, should_include_labels=True, use_filtered_positions=False, use_pandas_plotting=True)

# 🟢👁️‍🗨️ PhoKamran2023Paper Results

## Figure 1) pf1D Ratemaps, Active set, etc

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_1_add_replay_epoch_rasters


In [ ]:
# curr_active_pipeline.reload_default_display_functions()

pf1d_compare_graphics, (example_epoch_rasters_L, example_epoch_rasters_S), example_stacked_epoch_graphics, fig_1c_figures_out_dict = PAPER_FIGURE_figure_1_full(curr_active_pipeline) # did not display the pf1

In [ ]:
rdf = jonathan_firing_rate_analysis_result.rdf.rdf
rdf


## Figure 2) `PaperFigureTwo`: LxC/SxC Analyses
Note: this fails when SxC or LxC are empty for this session (as it's not meaningful to produce a comparison bar plot). In this case, aggregate across multiple sessions.

In [ ]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo

_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_fig_2.compute(curr_active_pipeline=curr_active_pipeline)
_out_fig_2.display()

## Figure 3) `PAPER_FIGURE_figure_3`: Firing Rate Index and Long/Short Firing Rate Replays v. Laps

In [ ]:
from neuropy.utils.matplotlib_helpers import FormattedFigureText
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies
# curr_active_pipeline.reload_default_display_functions()

_out, _out2 = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=False)

## Other Paper-related explorations:

In [ ]:
_out2.figures
_out.axes

In [ ]:

['start']


# Any change in duration over the recording session duration?
rdf['duration']

# rdf.plot.scatter('start', 'duration')

# rdf.plot.scatter('duration', 'num_neuron_participating') # strong positive trend

# rdf.plot.scatter('start', 'num_neuron_participating') # num neurons participating seems to increase over time, especially after the short track is introduced. Contrary to my hypothesis.

rdf.plot.scatter('start', 'num_short_only_neuron_participating')




# ['num_neuron_participating']

In [ ]:
curr_active_pipeline.reload_default_display_functions()
a_plot_obj = curr_active_pipeline.plot
a_plot_obj._display_spike_rasters_pyqtplot_2D()
# '_display_spike_rasters_pyqtplot_2D': ' Plots a standalone 2D raster plot\n        ',
# '_display_spike_rasters_pyqtplot_3D': ' Plots a standalone 3D raster plot with independent/standalone controls built-in\n        ',
# '_display_spike_rasters_pyqtplot_3D_with_2D_controls': ' Plots a standalone 3D raster plot (via pyqtgraph) with a separate 2D raster plot as the window with which you can adjust the viewed window. \n        ',
# '_display_spike_rasters_vedo_3D': ' Plots a standalone 3D raster plot with independent/standalone controls built-in\n        ',
# '_display_spike_rasters_vedo_3D_with_2D_controls': ' Plots a standalone 3D raster plot (via Vedo) with a separate 2D raster plot as the window with which you can adjust the viewed window. \n        ',
# '_display_spike_rasters_window': ' Displays a Spike3DRasterWindowWidget with a configurable set of raster widgets and controls in it.\n        ',type

In [ ]:
curr_active_pipeline.registered_display_function_docs_dict

In [ ]:
pf1d_compare_fig_L, pf1d_compare_fig_S = pf1d_compare_graphics.figures


In [ ]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import build_shared_sorted_neuronIDs
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph

ratemap = long_pf1D.ratemap
included_unit_neuron_IDs = EITHER_subset.track_exclusive_aclus
rediculous_final_sorted_all_included_neuron_ID, rediculous_final_sorted_all_included_pfmap = build_shared_sorted_neuronIDs(ratemap, included_unit_neuron_IDs, sort_ind=new_all_aclus_sort_indicies.copy())

In [ ]:
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(rediculous_final_sorted_all_included_pfmap, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
active_curves_sorted = long_pf1D.ratemap.normalized_tuning_curves[is_included][included_new_all_aclus_sort_indicies]
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(active_curves_sorted, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

## Stacked Epoch Plot
example_stacked_epoch_graphics = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', defer_render=False, save_figure=True)
pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers']
ax_L, ax_S = example_stacked_epoch_graphics.axes
final_figure_context_L, final_context_S = example_stacked_epoch_graphics.context

In [ ]:
## After launching the interactive Stacked Epoch Plots for user epoch selection, try to update the selection with previously added annotations:
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.GUI.Qt.Mixins.PaginationMixins import SelectionsObject
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

user_annotation_man = UserAnnotationsManager()
user_annotations = user_annotation_man.get_user_annotations()

final_context_L = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj')
final_context_S = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj')
# _out_pagination_controller.params.active_identifying_figure_ctx.adding_context(None,  user_annotation="selections")
selections_context_L = final_context_L.adding_context(None,  user_annotation="selections")
selections_context_S = final_context_S.adding_context(None,  user_annotation="selections")

saved_selection_L: SelectionsObject = pagination_controller_L.save_selection()
saved_selection_S: SelectionsObject = pagination_controller_S.save_selection()
# saved_selection_L = user_annotation_man.update_selections_from_annotations(saved_selection_L, user_annotations)
# saved_selection_S = user_annotation_man.update_selections_from_annotations(saved_selection_S, user_annotations)

saved_selection_L = saved_selection_L.update_selections_from_annotations(user_annotations, debug_print=False)
saved_selection_S = saved_selection_S.update_selections_from_annotations(user_annotations, debug_print=False)

## re-apply the selections:
pagination_controller_L.restore_selections(saved_selection_L, defer_render=True)
pagination_controller_S.restore_selections(saved_selection_S, defer_render=True)

ax_L[0].figure.canvas.draw()  # .figures.canvas.draw()
ax_S[0].figure.canvas.draw()  # .figures.canvas.draw()

# 2023-09-06 - Plot the decoded positions for each replay on the track:


In [ ]:
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import LinearTrackDimensions


long_track_dims = LinearTrackDimensions(track_length=100.0)
short_track_dims = LinearTrackDimensions(track_length=70.0)


# long_track_dims.plot_rects(ax)

In [ ]:
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import _test_LinearTrackDimensions_2D

app, w, cw, (long_track_dims, long_rect_items, long_rects), (short_track_dims, short_rect_items, short_rects) = _test_LinearTrackDimensions_2D()

In [ ]:
_out_fig_3_a, _out_fig_3_b = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=False)



In [ ]:
ax = _out_fig_3_a.axes[0]


In [ ]:
long_track_dims.plot_rects(ax)

In [ ]:
pagination_controller_L

In [ ]:
# Set the symbol properties
# symbol = pg.mkPen('w', width=1)  # Pen for the lines
size = 1.0  # Fixed x-width
symbol_brush = None  # No brush for the symbol (transparent fill)

# override_scatter_plot_kwargs = dict(pxMode=False, symbol='|', size=size, pen={'color': 'w', 'width': 1.0}) # , brush=symbol_brush

override_scatter_plot_kwargs = dict(pxMode=False, symbol='arrow_up', size=1.0, pen={'color': 'w', 'width': 1.0}, hoverable=True) # 

In [ ]:
filter_epoch_spikes_df_L.spikes.rebuild_fragile_linear_neuron_IDXs();
filter_epoch_spikes_df_S.spikes.rebuild_fragile_linear_neuron_IDXs();

In [ ]:
app_L, win_L, plots_L, plots_data_L = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
										epoch_id_key_name='replay_epoch_id', scatter_app_name="Long Decoded Example Replays")

In [ ]:
app_S, win_S, plots_S, plots_data_S = plot_multiple_raster_plot(epochs_df_S, filter_epoch_spikes_df_S, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                 epoch_id_key_name='replay_epoch_id', scatter_app_name="Short Decoded Example Replays")

In [ ]:
# Test single `plot_raster_plot` calls
an_epoch = list(epochs_df_L.itertuples())[0]
an_epoch_spikes_df = filter_epoch_spikes_df_L[filter_epoch_spikes_df_L['replay_epoch_id'] == an_epoch.Index]

_out_single_raster_plot = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test1")
_out_single_raster_plot2 = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test2")

In [ ]:
app_alt, win_alt, plots_alt, plots_data_alt = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                         epoch_id_key_name='replay_epoch_id', scatter_app_name="ALT Long Decoded Example Replays")

### Testing `plot_kourosh_activity_style_figure` for debugging:

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.helpers import _helper_make_scatterplot_clickable

plot_aclus = EITHER_subset.track_exclusive_aclus.copy()
# plot_aclus = EITHER_subset.track_exclusive_aclus[new_all_aclus_sort_indicies].copy()
_out_A = plot_kourosh_activity_style_figure(long_results_obj, long_session, plot_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=13, callout_epoch_IDXs=None, skip_rendering_callouts=False)

In [ ]:
app, win, plots, plots_data = _out_A
# plots

In [ ]:
#TODO 2023-06-27 10:42: - [ ] Desperitely need a class that "explodes" the important variables and their types out of a DynamicParameters (dict-like) or other object.


In [ ]:
_out_n = plot_kourosh_activity_style_figure(long_results_obj, long_session, EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=49, callout_epoch_IDXs=np.arange(6), skip_rendering_callouts=False)

# 2023-07-14 - LxC and SxC PhoJonathanSession plots

In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import BatchPhoJonathanFiguresHelper

## Get global 'jonathan_firing_rate_analysis' results:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis.neuron_replay_stats_df, curr_jonathan_firing_rate_analysis.rdf.rdf, curr_jonathan_firing_rate_analysis.rdf.aclu_to_idx, curr_jonathan_firing_rate_analysis.irdf.irdf

# with VizTracer(output_file=f"viztracer_display_BatchPhoJonathanFiguresHelper_PlusPDF_20.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# active_out_figures_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}
# print(f'active_out_figures_dict: {active_out_figures_dict}')

# BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset
active_out_figures_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=EITHER_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}
print(f'active_out_figures_dict: {active_out_figures_dict}')



In [ ]:
# 2023-09-07 - Build Example LxC/SxC cells from handpicked examples: aclus = [4, 58]
# from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import build_extra_cell_info_label_string
curr_active_pipeline.reload_default_display_functions()
_out2 = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', n_max_plot_rows=2, save_figure=False, included_unit_neuron_IDs=[4, 58]) # , included_unit_neuron_IDs=[4, 58]


In [ ]:
# curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=long_epoch_context)


# curr_active_pipeline.display(display_function='_display_placemaps_pyqtplot_2D', active_session_configuration_context=long_epoch_context)
curr_active_pipeline.display(display_function='_display_1d_placefields', active_session_configuration_context=long_epoch_context)
# curr_active_pipeline.display(display_function='_display_2d_placefield_occupancy', active_session_configuration_context=long_epoch_context)



In [ ]:
curr_active_pipeline.display(display_function='_display_2d_placefield_occupancy', active_session_configuration_context=long_epoch_context)


In [ ]:
curr_active_pipeline.display(display_function='_display_2d_placefield_occupancy', active_session_configuration_context=short_epoch_context)


In [ ]:
curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=short_epoch_context)


In [ ]:
curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=long_epoch_context)

In [ ]:
fig = plt.figure() # new figure to hold the result
# can show the figures by looping through and calling
for a_ctxt, a_fig in active_out_figures_dict.items():
    print(f'showing: {a_ctxt}')
    a_fig.show()
    


In [ ]:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]] # only uses global_session
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)

long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_laps, long_replays, short_laps, short_replays, global_laps, global_replays = [long_short_fr_indicies_analysis_results[k] for k in ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays']]


In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
long_exclusive.track_exclusive_df

In [ ]:
long_exclusive.track_exclusive_df['lap_delta_minus_inst_fr'] = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )
long_exclusive.track_exclusive_df['lap_delta_plus_inst_fr'] = LxC_ThetaDeltaPlus.cell_agg_inst_fr_list # (n_cells, )
long_exclusive.track_exclusive_df


In [ ]:
# d = LxC_ThetaDeltaMinus.epoch_agg_inst_fr_list # (n_epochs, n_cells)
# d = d[:,1]
d = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )
d
long_exclusive.track_exclusive_df['lap_delta_minus_inst_fr'] = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )


In [ ]:
## Drop out the very low (inactive) Epochs... I guess? But we want it to be influenced by the inactive epochs.
curr_active_pipeline.	

In [ ]:

len(LxC_ThetaDeltaMinus.inst_fr_signals_list) # n_epochs
d = LxC_ThetaDeltaMinus.inst_fr_df_list
d[0]
# (n_epochs, n_cells)
# print(f'{d.shape}')

## NOW: 2023-07-11 - Testing Batch-computed inst_firing_rates

In [ ]:
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations


## Load the saved across-session results:
inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=0) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

# `active_pf_nD`, `active_pf_nD_dt` visualizations

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedOccupancyPlotter import TimeSynchronizedOccupancyPlotter



In [ ]:

curr_sync_occupancy_plotter = TimeSynchronizedOccupancyPlotter(global_pf2D_dt)
curr_sync_occupancy_plotter.show()

# 2023-07-07 - `batch_extended_programmatic_figures` Testing

In [ ]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=False)

In [ ]:
%pdb off
%load_ext viztracer
from viztracer import VizTracer
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str

In [ ]:

with VizTracer(output_file=f"viztracer_{get_now_time_str()}-batch_extended_programmatic_figures.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
    batch_extended_programmatic_figures(curr_active_pipeline, write_vector_format=False, write_png=False, debug_print=False)

In [ ]:
batch_extended_programmatic_figures(curr_active_pipeline, write_vector_format=False, write_png=False, debug_print=False)

In [ ]:
np.sum(np.logical_or(neuron_replay_stats_df['is_refined_LxC'], neuron_replay_stats_df['is_refined_SxC']))

In [ ]:
JonathanFiringRateAnalysisResult = None

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)


In [ ]:
jonathan_firing_rate_analysis_result

In [ ]:
# Relative Entropy/Surprise Results:
active_extended_stats = global_results['extended_stats']
active_relative_entropy_results = active_extended_stats['pf_dt_sequential_surprise'] # DynamicParameters
historical_snapshots = active_relative_entropy_results['historical_snapshots']
post_update_times: np.ndarray = active_relative_entropy_results['post_update_times'] # (4152,) = (n_post_update_times,)
snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
time_intervals: np.ndarray = active_relative_entropy_results['time_intervals']
surprise_time_bin_duration = (post_update_times[2]-post_update_times[1])
long_short_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['long_short_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
short_long_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['short_long_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
flat_relative_entropy_results: np.ndarray = active_relative_entropy_results['flat_relative_entropy_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_results: np.ndarray = active_relative_entropy_results['flat_jensen_shannon_distance_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_across_all_positions: np.ndarray = np.sum(np.abs(flat_jensen_shannon_distance_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)
flat_surprise_across_all_positions: np.ndarray = np.sum(np.abs(flat_relative_entropy_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)

## Get the placefield dt matrix:
if 'snapshot_occupancy_weighted_tuning_maps' not in active_relative_entropy_results:
	## Compute it if missing:
	occupancy_weighted_tuning_maps_over_time = np.stack([placefield_snapshot.occupancy_weighted_tuning_maps_matrix for placefield_snapshot in historical_snapshots.values()])
	active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] = occupancy_weighted_tuning_maps_over_time
else:
	occupancy_weighted_tuning_maps_over_time = active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] # (n_post_update_times, n_neurons, n_xbins)


In [ ]:
# Time-dependent
long_pf1D_dt, short_pf1D_dt, global_pf1D_dt = long_results.pf1D_dt, short_results.pf1D_dt, global_results.pf1D_dt
long_pf2D_dt, short_pf2D_dt, global_pf2D_dt = long_results.pf2D_dt, short_results.pf2D_dt, global_results.pf2D_dt
global_pf1D_dt: PfND_TimeDependent = global_results.pf1D_dt
global_pf2D_dt: PfND_TimeDependent = global_results.pf2D_dt

In [ ]:
## long_short_endcap_analysis: checks for cells localized to the endcaps that have their placefields truncated after shortening the track
truncation_checking_result: TruncationCheckingResults = curr_active_pipeline.global_computation_results.computed_data.long_short_endcap
disappearing_endcap_aclus = truncation_checking_result.disappearing_endcap_aclus
# disappearing_endcap_aclus
trivially_remapping_endcap_aclus = truncation_checking_result.minor_remapping_endcap_aclus
# trivially_remapping_endcap_aclus
significant_distant_remapping_endcap_aclus = truncation_checking_result.significant_distant_remapping_endcap_aclus
# significant_distant_remapping_endcap_aclus
appearing_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[jonathan_firing_rate_analysis_result.neuron_replay_stats_df['track_membership'] == SplitPartitionMembership.RIGHT_ONLY].index
# appearing_aclus

# 2023-09-26 - Interactive `napari` testing

In [ ]:
import napari
from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_add_surprise_data_layers, napari_add_animal_position, napari_set_time_windw_index

# create the viewer and window
viewer = napari.Viewer()
napari_add_surprise_data_layers(viewer, active_relative_entropy_results)
napari_add_animal_position(viewer=viewer, position_df=global_pf1D_dt.all_time_filtered_pos_df[['t','x','binned_x']], time_intervals=time_intervals)

In [ ]:
drop_below_threshold:float=0.0000001
active_time_dependent_placefields = deepcopy(global_pf1D_dt)
curr_ratemap = active_time_dependent_placefields.ratemap
images = curr_ratemap.tuning_curves.copy() # 1D: (82, 112), 2D:(43, 63, 63)
print(f'images.shape: {np.shape(images)}')
occupancy = curr_ratemap.occupancy
print(f'occupancy: {occupancy.shape}')
# Compute Images:
included_unit_indicies = np.arange(np.shape(images)[0]) # include all unless otherwise specified
nMapsToShow = len(included_unit_indicies)
print(f'nMapsToShow: {nMapsToShow}')
# Get single cell info:
# neuron_IDX = curr_included_unit_index
# cell_ID = active_time_dependent_placefields.ratemap.neuron_ids[neuron_IDX]
# curr_cell_identifier_string = f'Cell[{cell_ID}]'
# curr_plot_identifier_string = f'TimeSynchronizedPlacefieldsPlotter.{curr_cell_identifier_string}'

# Build the image item:
# Update the image:
# image = np.squeeze(images[a_linear_index,:,:])
image = np.squeeze(images)
# Pre-filter the data:
with np.errstate(divide='ignore', invalid='ignore'):
	image = np.array(image) / np.nanmax(image) # note scaling by maximum here!
	if drop_below_threshold is not None:
		image[:, np.where(occupancy < drop_below_threshold)] = np.nan # null out the occupancy

image.shape

In [ ]:
from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_export_video_frames



In [ ]:
type(screenshot)
screenshot.shape # (1165, 1488, 4)

# 2023-09-27 - Exporting surprise ndarray to a video file for visualization!

In [ ]:

# Add the animal's closest current position (binned) as a single colored pixel along the top of the array:
len(global_pf1D_dt.historical_snapshots)

global_pf1D_dt.historical_snapshots

In [ ]:
import cv2
from pyphocorehelpers.plotting.video_output_helpers import save_array_as_video

colormap = cv2.COLORMAP_JET
array = active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'].copy()
color_array = cv2.applyColorMap(array, colormap)
video_out_path = save_array_as_video(array=color_array, video_filename='output/videos/snapshot_occupancy_weighted_tuning_maps.avi', isColor=True)
print(f'video_out_path: {video_out_path}')
reveal_in_system_file_manager(video_out_path)

In [ ]:
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import export_active_relative_entropy_results_videos

video_output_parent_path = export_active_relative_entropy_results_videos(active_relative_entropy_results, active_context=curr_active_pipeline.get_session_context())
reveal_in_system_file_manager(video_output_parent_path)

# 2023-09-26 - Interactive `Spike3DRasterWindowWidget` exploration`

In [ ]:

# Gets the existing SpikeRasterWindow or creates a new one if one doesn't already exist:
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper
import pyphoplacecellanalysis.External.pyqtgraph as pg # Used to get the app for TopLevelWindowHelper.top_level_windows
## For searching with `TopLevelWindowHelper.all_widgets(...)`:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget

found_spike_raster_windows = TopLevelWindowHelper.all_widgets(pg.mkQApp(), searchType=Spike3DRasterWindowWidget)

if len(found_spike_raster_windows) < 1:
	# no existing spike_raster_windows. Make a new one
	print(f'no existing SpikeRasterWindow. Creating a new one.')
	# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
	active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D().values()

else:
	print(f'found {len(found_spike_raster_windows)} existing Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...). Will use the most recent.')
	# assert len(found_spike_raster_windows) == 1, f"found {len(found_spike_raster_windows)} Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...) but require exactly one."
	# Get the most recent existing one and reuse that:
	spike_raster_window = found_spike_raster_windows[0]




In [ ]:
Spike3DRasterWindowWidget.

# Interactive SpikeRasterWindow

In [ ]:
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot

global_pf1D_dt: PfND_TimeDependent = global_results.pf1D_dt
global_pf2D_dt: PfND_TimeDependent = global_results.pf2D_dt


In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedOccupancyPlotter import TimeSynchronizedOccupancyPlotter
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.Mixins.helpers import build_combined_time_synchronized_plotters_window

active_pf_2D_dt = global_pf2D_dt
active_pf_2D_dt.reset()
active_pf_2D_dt.update(t=500.0, start_relative_t=True)
all_plotters, root_dockAreaWindow, app = build_combined_time_synchronized_plotters_window(active_pf_2D_dt, fixed_window_duration = 15.0)
controlling_widget, curr_sync_occupancy_plotter, curr_placefields_plotter = all_plotters # end up in some sort of `linkedViewChanged` loop

In [ ]:
# Draw a VERY simple and efficient 1D plot of the animal on the track.


In [ ]:
controlling_widget

In [ ]:
# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
_out = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()

In [ ]:
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper
import pyphoplacecellanalysis.External.pyqtgraph as pg # Used to get the app for TopLevelWindowHelper.top_level_windows
## For searching with `TopLevelWindowHelper.all_widgets(...)`:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget

found_spike_raster_windows = TopLevelWindowHelper.all_widgets(pg.mkQApp(), searchType=Spike3DRasterWindowWidget)
# assert len(found_spike_raster_windows) == 1, f"found {len(found_spike_raster_windows)} Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...) but require exactly one."
if len(found_spike_raster_windows) > 1:
	print(f'found {len(found_spike_raster_windows)} widgets with the searchType provided:')
	print(f'{found_spike_raster_windows}')
	print(f'defaulting to using the most recent [-1]...')
spike_raster_window = found_spike_raster_windows[-1]

# Extras:
active_2d_plot = spike_raster_window.spike_raster_plt_2d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>
active_3d_plot = spike_raster_window.spike_raster_plt_3d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>

In [ ]:
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState

active_pf_2D = global_pf2D
## Example 1: De-emphasize spikes excluded from the placefield calculations:
is_spike_included_in_pf = np.isin(spike_raster_window.spike_raster_plt_2d.spikes_df.index, active_pf_2D.filtered_spikes_df.index)
spike_raster_window.spike_raster_plt_2d.update_spike_emphasis(np.logical_not(is_spike_included_in_pf), SpikeEmphasisState.Deemphasized)


In [ ]:
is_spike_included_in_pf

In [ ]:
active_pf_2D_dt.update(t=1000.0, start_relative_t=True)

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter

curr_sync_occupancy_plotter = TimeSynchronizedOccupancyPlotter(global_pf2D_dt)
curr_sync_occupancy_plotter.show()

In [ ]:
global_pf2D_dt.historical_snapshots

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter

curr_sync_placefield_plotter = TimeSynchronizedPlacefieldsPlotter(global_pf2D_dt)
curr_sync_placefield_plotter.show()

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter

curr_sync_placefield_plotter1D = TimeSynchronizedPlacefieldsPlotter(global_pf1D_dt)
curr_sync_placefield_plotter1D.show()

# 2023-07-19 - Validation with 3D tools

In [ ]:
import pyvista as pv

display_output = {}
# active_config_name = long_epoch_name
active_config_name = global_epoch_name
active_config = curr_active_pipeline.active_configs[active_config_name]
active_config.plotting_config.should_use_linear_track_geometry = True # indicate that it's a linear track so the better geometry can be used

# 3D Plotters

## 📣 Programmatically adding several epoch rectangles by calling the addRenderable context menu functions all at once for SpikeRaster2D

In [ ]:
add_renderables_menu = active_2d_plot.ui.menus.custom_context_menus.add_renderables[0].programmatic_actions_dict
menu_commands = ['AddTimeIntervals.PBEs', 'AddTimeIntervals.Ripples', 'AddTimeIntervals.Replays', 'AddTimeIntervals.Laps', 'AddTimeIntervals.Session.Epochs']
for a_command in menu_commands:
    add_renderables_menu[a_command].trigger()

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import General2DRenderTimeEpochs, Ripples_2DRenderTimeEpochs, inline_mkColor
## Inline Concise: Position Replays, PBEs, and Ripples all below the scatter:
# active_2d_plot.interval_datasources.Replays.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-10.0, height=7.5, pen_color=inline_mkColor('orange', 0.8), brush_color=inline_mkColor('orange', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.PBEs.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-2.0, height=1.5, pen_color=inline_mkColor('pink', 0.8), brush_color=inline_mkColor('pink', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.Ripples.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.SessionEpochs .update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline
epochs_update_dict = {
    'Replays':dict(y_location=-10.0, height=7.5, pen_color=inline_mkColor('orange', 0.8), brush_color=inline_mkColor('orange', 0.5)),
    'PBEs':dict(y_location=-2.0, height=1.5, pen_color=inline_mkColor('pink', 0.8), brush_color=inline_mkColor('pink', 0.5)),
    'Ripples':dict(y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5)),
    'SessionEpochs ':dict(y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5)),
}
active_2d_plot.update_rendered_intervals_visualization_properties(epochs_update_dict)


In [ ]:
rendered_interval_keys = list(interval_info.keys())
desired_interval_height_ratios = [2.0, 2.0, 1.0, 0.1, 1.0, 1.0, 1.0] # ratio of heights to each interval
required_vertical_offsets, required_interval_heights = EpochRenderingMixin.build_stacked_epoch_layout(desired_interval_height_ratios, epoch_render_stack_height=20.0, interval_stack_location='below')
stacked_epoch_layout_dict = {interval_key:dict(y_location=y_location, height=height) for interval_key, y_location, height in zip(rendered_interval_keys, required_vertical_offsets, required_interval_heights)} # Build a stacked_epoch_layout_dict to update the display
active_2d_plot.update_rendered_intervals_visualization_properties(stacked_epoch_layout_dict)

## 🪟 ipcDataExplorer - 3D Interactive Tuning Curves Plotter

In [ ]:
curr_active_pipeline.registered_display_function_docs_dict

'_display_3d_image_plotter'
'_display_long_short_laps'
'_display_3d_interactive_custom_data_explorer'


In [ ]:
# from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractiveDataExplorerBase import InteractiveDataExplorerBase
# from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractiveCustomDataExplorer import InteractiveCustomDataExplorer # TypeError: super(type, obj): obj must be an instance or subtype of type

active_config_name = global_epoch_context
# curr_active_pipeline.reload_default_display_functions()
# _out = curr_active_pipeline.display('_display_3d_interactive_custom_data_explorer', long_epoch_name) # long_epoch_context
display_dict = curr_active_pipeline.display('_display_3d_interactive_custom_data_explorer', active_config_name) # does not work, missing color info?
iplapsDataExplorer = display_dict['iplapsDataExplorer']
# plotter is available at
p = display_dict['plotter']
iplapsDataExplorer


In [ ]:
import ipywidgets as widgets

# df = pd.DataFrame([curr_active_pipeline.registered_display_function_docs_dict])
# df = curr_active_pipeline.registered_display_function_docs_dict
df = pd.DataFrame.from_dict(curr_active_pipeline.registered_display_function_docs_dict, orient='index', columns=['Value'])
display_widget = widgets.Output()
with display_widget:
	display(df)
display_widget

In [ ]:
from neuropy.utils.matplotlib_helpers import plot_position_curves_figure

curr_active_pipeline.reload_default_display_functions()

In [ ]:
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
graphics_output_dict = curr_active_pipeline.display('_display_long_short_laps', active_identifying_session_ctx)

In [ ]:
_out3 = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices')

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_3d
# ## single_combined_plot == True mode (mode 1.):
# p, laps_pages = plot_lap_trajectories_3d(curr_active_pipeline.sess, single_combined_plot=True)
# p.show()

## single_combined_plot == False mode (mode 2.):        
p, laps_pages = plot_lap_trajectories_3d(curr_active_pipeline.sess, single_combined_plot=False, curr_num_subplots=len(curr_active_pipeline.sess.laps.lap_id), active_page_index=0)
p.show()

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_2d

p_laps_2D, axs_laps_2D, laps_2D_pages = plot_lap_trajectories_2d(curr_active_pipeline.sess, curr_num_subplots=5, active_page_index=0)

In [ ]:
p_laps_2D.suptitle('plot_lap_trajectories_2d')


In [ ]:
ax = axs_laps_2D[0,0]
ax_empty = axs_laps_2D[-1,-1]

In [ ]:
long_track_position_offset = 1.0 * long_track_dims.compute_position_offset(grid_bin_bounds=grid_bin_bounds) # array([49.43, 140.61])
long_track_position_offset

In [ ]:
grid_bin_bounds_center_point = (point_tuple_mid_point(grid_bin_bounds[0]), point_tuple_mid_point(grid_bin_bounds[1])) # (145.43, 140.61)
print(f'grid_bin_bounds_center_point: {grid_bin_bounds_center_point}')
x_midpoint, y_midpoint = grid_bin_bounds_center_point

In [ ]:
user_annotation_man = UserAnnotationsManager()
user_annotations = user_annotation_man.get_user_annotations()

final_context_L = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj')
final_context_S = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj')
# _out_pagination_controller.params.active_identifying_figure_ctx.adding_context(None,  user_annotation="selections")
selections_context_L = final_context_L.adding_context(None,  user_annotation="selections")
selections_context_S = final_context_S.adding_context(None,  user_annotation="selections")

## try to get the user annotations for this session:
try:
	selection_idxs_L = user_annotations[selections_context_L]
	selection_idxs_S = user_annotations[selections_context_S]
except KeyError as e:
	if allow_interactive_selection:
		print(f'user annotations <good replay selections> are not found. Creating them interactively...')
		user_annotations = interactive_good_epoch_selections(annotations_man=user_annotation_man, curr_active_pipeline=curr_active_pipeline)  # perform interactive selection. Should block here.
		selection_idxs_L = user_annotations[selections_context_L]
		selection_idxs_S = user_annotations[selections_context_S]
	else:
		print(f'interactive annotation is not permitted. Failing.')
		raise e
except Exception as e:
	print('Unhandled exception: {e}')
	raise

# for updating the filter_epochs_df (`filter_epochs_df`) from the selections:
self.filter_epochs_df['long_is_user_included'] = np.isin(self.filter_epochs_df.index, selection_idxs_L)
self.filter_epochs_df['short_is_user_included'] = np.isin(self.filter_epochs_df.index, selection_idxs_S)

In [ ]:
%pdf off

In [ ]:
from copy import deepcopy
import numpy as np
from neuropy.utils.mixins.binning_helpers import transition_matrix

### 1D Transition Matrix:

def _compute_position_transition_matrix(xbin_labels, binned_x: np.ndarray, n_powers:int=3):
	"""  1D Transition Matrix from binned positions (e.g. 'binned_x')

		pf1D.xbin_labels # array([  1,   2,   3,   4,  ...)
		pf1D.filtered_pos_df['binned_x'].to_numpy() # array([116, 115, 115, ...,  93,  93,  93], dtype=int64)
	"""
	num_position_states = len(xbin_labels)
	# binned_x = pos_1D.to_numpy()
	binned_x_indicies = binned_x - 1
	binned_x_transition_matrix = transition_matrix(deepcopy(binned_x_indicies), markov_order=1, max_state_index=num_position_states)
	# binned_x_transition_matrix_higher_order_list = [binned_x_transition_matrix, transition_matrix(deepcopy(binned_x_indicies), markov_order=2, max_state_index=num_position_states), transition_matrix(deepcopy(binned_x_indicies), markov_order=3, max_state_index=num_position_states)]

	binned_x_transition_matrix[np.isnan(binned_x_transition_matrix)] = 0.0
	binned_x_transition_matrix_higher_order_list = [binned_x_transition_matrix] + [np.linalg.matrix_power(binned_x_transition_matrix, n) for n in np.arange(2, n_powers+1)]
	# , np.linalg.matrix_power(binned_x_transition_matrix, 2), np.linalg.matrix_power(binned_x_transition_matrix, 3)
	# binned_x_transition_matrix.shape # (64, 64)
	return binned_x_transition_matrix_higher_order_list

def _build_decoded_positions_transition_matrix(active_one_step_decoder):
	""" Compute the transition_matrix from the decoded positions 

	TODO: make sure that separate events (e.g. separate replays) are not truncated creating erronious transitions

	"""
	# active_time_window_variable = active_one_step_decoder.time_window_centers # get time window centers (n_time_window_centers,) # (4060,)
	# active_most_likely_positions = active_one_step_decoder.most_likely_positions.T # (4060, 2) NOTE: the most_likely_positions for the active_one_step_decoder are tranposed compared to the active_two_step_decoder
	# active_most_likely_positions = active_two_step_decoder.most_likely_positions # (2, 4060)
	active_one_step_decoder.most_likely_position_flat_indicies
	# active_most_likely_positions = active_one_step_decoder.revised_most_likely_positions.T
	# active_most_likely_positions #.shape # (36246,)

	most_likely_position_indicies = np.squeeze(np.array(np.unravel_index(active_one_step_decoder.most_likely_position_flat_indicies, active_one_step_decoder.original_position_data_shape))) # convert back to an array
	most_likely_position_xbins = most_likely_position_indicies + 1 # add 1 to convert back to a bin label from an index
	# most_likely_position_indicies # (1, 36246)

	xbin_labels = np.arange(active_one_step_decoder.original_position_data_shape[0]) + 1

	decoded_binned_x_transition_matrix_higher_order_list = _compute_position_transition_matrix(xbin_labels, most_likely_position_indicies)
	return decoded_binned_x_transition_matrix_higher_order_list, xbin_labels


# pf1D = deepcopy(curr_active_pipeline.computation_results['maze1'].computed_data['pf1D'])
pf1D = deepcopy(global_pf1D)
# pf1D = deepcopy(short_pf1D)
# pf1D = deepcopy(long_pf1D)
binned_x_transition_matrix_higher_order_list = _compute_position_transition_matrix(pf1D.xbin_labels, pf1D.filtered_pos_df['binned_x'].to_numpy())

In [ ]:
pf1D.filtered_pos_df.plot(x='t',y='x')

In [ ]:
# Visualization ______________________________________________________________________________________________________ #
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability
out = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[0], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix', title="Transition Matrix for binned x (from, to)", variable_label='Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)
# out.add_data(row=1, col=0, matrix=active_eloy_analysis.pf_overlapDensity_2D, xbins=active_pf_2D_dt.xbin_labels, ybins=active_pf_2D_dt.ybin_labels, name='pf_overlapDensity', title='pf overlapDensity metric', variable_label='pf overlapDensity')

# out.add_data(row=1, col=0, matrix=binned_x_transition_matrix_higher_order_list[1], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^2', title='2nd Order Transition Matrix for binned x (from, to)', variable_label='2nd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE
# out.add_data(row=2, col=0, matrix=binned_x_transition_matrix_higher_order_list[2], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^3', title='3rd Order Transition Matrix for binned x (from, to)', variable_label='3rd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE

# Separate Windows for each:
# out2 = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[1], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix^2', title="2nd Order Transition Matrix for binned x (from, to)", variable_label='2nd Order Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)
# out3 = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[2], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix^3', title="3rd Order Transition Matrix for binned x (from, to)", variable_label='3rd Order Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)

## 1. Compute the transition_matrix from the decoded positions


In [ ]:
# BayesianPlacemapPositionDecoder
# active_one_step_decoder = deepcopy(long_one_step_decoder_1D) # long_results_obj.original_1D_decoder
long_decoded_binned_x_transition_matrix_higher_order_list, long_xbin_labels = _build_decoded_positions_transition_matrix(active_one_step_decoder=deepcopy(long_one_step_decoder_1D))
short_decoded_binned_x_transition_matrix_higher_order_list, short_xbin_labels = _build_decoded_positions_transition_matrix(active_one_step_decoder=deepcopy(short_one_step_decoder_1D))


In [ ]:
out_decoded = BasicBinnedImageRenderingWindow(decoded_binned_x_transition_matrix_higher_order_list[0], active_one_step_decoder.xbin_centers, active_one_step_decoder.xbin_centers, name='decoded_binned_x_transition_matrix', title="DECODED Transition Matrix for binned x (from, to)", variable_label='Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)


In [ ]:
i = 0
out = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[i], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix', title="Transition Matrix for binned x (from, to)", variable_label='Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)
out.add_data(row=1, col=0, matrix=long_decoded_binned_x_transition_matrix_higher_order_list[i], xbins=long_xbin_labels, ybins=long_xbin_labels, name='long_decoded_binned_x_transition_matrix', title='Long DECODED Transition Matrix', variable_label='long decoded')
out.add_data(row=2, col=0, matrix=short_decoded_binned_x_transition_matrix_higher_order_list[i], xbins=short_xbin_labels, ybins=short_xbin_labels, name='short_decoded_binned_x_transition_matrix', title='Short DECODED Transition Matrix', variable_label='short decoded')
# out.add_data(row=1, col=0, matrix=binned_x_transition_matrix_higher_order_list[1], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^2', title='2nd Order Transition Matrix for binned x (from, to)', variable_label='2nd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE
# out.add_data(row=2, col=0, matrix=binned_x_transition_matrix_higher_order_list[2], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^3', title='3rd Order Transition Matrix for binned x (from, to)', variable_label='3rd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE


In [ ]:
active_one_step_decoder.most_likely_position_indicies

In [ ]:

## 2. Use the position transition matrix to determine how likely each decoded position's transition is

In [ ]:
time_binned_instantaneous_unit_specific_spike_rate = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.time_binned_instantaneous_unit_specific_spike_rate
timestamps = time_binned_instantaneous_unit_specific_spike_rate.time_bins

value_df = time_binned_instantaneous_unit_specific_spike_rate.instantaneous_unit_specific_spike_rate_values
value_df

In [ ]:
# Number of spikes version:
time_binned_unit_specific_spike_rate = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.time_binned_unit_specific_spike_rate
timestamps = time_binned_unit_specific_spike_rate.time_bins
value_df = time_binned_unit_specific_spike_rate.time_binned_unit_specific_binned_spike_rate
value_df

# global_computations pickling experiment
global_computation_result has some representation of the global functions? Noticed that changing one of the `validate_computation_test` for a function resulted in the resultant pipeline being unpicklable although the data I added wasn't an issue.

```python
@function_attributes(short_name='long_short_fr_indicies_analyses', tags=['short_long','firing_rate', 'computation'], input_requires=[], output_provides=['long_short_fr_indicies_analysis'], uses=['pipeline_complete_compute_long_short_fr_indicies'], used_by=[], creation_date='2023-04-11 00:00', 
                         validate_computation_test=lambda curr_active_pipeline, computation_filter_name='maze': (curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis'], curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']['long_short_fr_indicies_df']), is_global=True)
```

### //2023-9-29 2:48 AM: Confirmed pickling issue with global_computation_result due to storing functions implicitly in `.computation_times`
Not sure whether to convert to just use `a_fn.__name__` or some other non-changing identifier. The function is never used from the keys, I basically just want it to be a dynamic reference to the current version of the function although it might be useful to be able to see if the function has been modified in any way since last compute.

[/c:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Model/ComputationResults.py:124](vscode://file/c:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Model/ComputationResults.py:124)
```python
computation_times = state.pop('computation_times', None)
        if computation_times is not None:
            ## Strip the parameter-ness of the items:
            computation_times = {str(fn.__name__):a_time for fn, a_time in computation_times.items()} # extract just the name from the function.
            state['computation_times'] = computation_times # add back in

```

In [ ]:
import dill

_test_obj = curr_active_pipeline.global_computation_results


computation_times=DynamicParameters({<function LongShortTrackComputations._perform_long_short_decoding_analyses at 0x000001C818E06AF0>: datetime.datetime(2023, 9, 11, 11, 19, 49, 530975), <function LongShortTrackComputations._perform_long_short_firing_rate_analyses at 0x000001C818E06DC0>: datetime.datetime(2023, 9, 11, 11, 20, 40, 45474), <function LongShortTrackComputations._perform_jonathan_replay_firing_rate_analyses at 0x000001C818E06F70>: datetime.datetime(2023, 9, 11, 11, 20, 41, 5600), <function LongShortTrackComputations._perform_long_short_post_decoding_analysis at 0x000001C8194DB310>: datetime.datetime(2023, 9, 11, 11, 20, 43, 767961), <function LongShortTrackComputations._perform_long_short_endcap_analysis at 0x000001C8194DB5E0>: datetime.datetime(2023, 9, 19, 8, 24, 52, 891582), <function LongShortTrackComputations._perform_long_short_pf_overlap_analyses at 0x000001C8194DB820>: datetime.datetime(2023, 9, 28, 20, 32, 8, 468427), <function LongShortTrackComputations._perform_long_short_firing_rate_analyses at 0x000001C8194DB9D0>: datetime.datetime(2023, 9, 28, 20, 33, 4, 411283), <function LongShortTrackComputations._perform_jonathan_replay_firing_rate_analyses at 0x000001C8194DBB80>: datetime.datetime(2023, 9, 28, 20, 33, 5, 809308)}))]

In [ ]:
_test_obj.computation_times

In [ ]:
# dill.detect.badtypes(_test_obj) # pyphoplacecellanalysis.General.Model.ComputationResults.ComputationResult
dill.detect.baditems(_test_obj)

